In [3]:
import pandas as pd
import numpy as np

In [4]:
filenames = ['case note - unformatted',
            'Class and Testing History - unformatted',
            'Client Community Activity - unformatted',
            'Client information (demographics and assessment) - unformatted',
            'DDA Case info - unformatted','Employer contact history - unformatted',
            'Employer placement hours - unformatted',
            'Employer placement record - unformatted',
            'Funding - unformatted',
            'guardianship information - unformatted',
            'individual work schedule (eval) - unformatted',
            'Intake Wizard (intake application) - unformatted','Mentor availability - unformatted',
            'outcome activity (case note) - unformatted',
            'outcome area (case notes) - unformatted',
            'outcome goal (goals) - unformatted',
            'people - unformatted',
            'profiles - unformatted',
            'profiles - users - unformatted',
            'Relationships (contacts) - unformatted',
            'contacts - unformatted']

In [13]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [159]:
pd.reset_option('display.max_columns', None)
pd.reset_option('display.max_rows', None)

people is the highest source of truth for participants the ID associated with people is the id used in the individual

people also holds contacts, the individual is the type for participants

### This is the start of merging the individual people information into one big spreadsheet

In [5]:
file1 = pd.read_excel(r'redacted file path\people - unformatted.xlsx')
file1 = file1.add_prefix('people_')
file2 = pd.read_excel(r'redacted file path\Client information (demographics and assessment) - unformatted.xlsx')
file2 = file2.add_prefix('demo_')

In [279]:
file1.columns

Index(['people_ID', 'people_First Name', 'people_Full Name', 'people_Title',
       'people_Forms Dropdowns', 'people_Created', 'people_Created By',
       'people_Content Type', 'people_E-Mail', 'people_E-mail 2',
       'people_Phone', 'people_Mobile Number', 'people_Professional Type',
       'people_Comments', 'people_Filter', 'people_Folder Child Count',
       'people_Item Child Count', 'people_Last Name', 'people_Modified',
       'people_Modified By', 'people_Item Type', 'people_Path'],
      dtype='object')

In [6]:
merge1 = file1.merge(file2,how='left',left_on='people_Full Name',right_on='demo_Individual')
merge2 = merge1[merge1['people_Forms Dropdowns']=='Individual']

In [7]:
# merge2.to_excel(r'redacted file path\people merge.xlsx')

In [8]:
file3 = pd.read_excel(r'redacted file path\profiles - unformatted.xlsx')
file3 = file3.add_prefix('prof_')
file4 = pd.read_excel(r'redacted file path\Intake Wizard (intake application) - unformatted.xlsx')
file4 = file4.add_prefix('intake_')

In [9]:
merge3 = merge2.merge(file3,how='left',left_on='people_Full Name',right_on='prof_Person')
# merge3 = merge3.drop(columns=['ID_x','Title','Forms Dropdowns','Created By','Content Type_x','ID_y','Content Type_y','View_x','View_y','Item Type_x','Item Type_y','Path','Path_x','Path_y','Item Type','Created_x','Created By_x','E-Mail_x'])
# merge3.head(2)

In [10]:
# merge3.columns

In [11]:
merge4 = merge3.merge(file4,how='left',left_on='people_Full Name',right_on='intake_Full Name')
merge4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1532 entries, 0 to 1531
Columns: 257 entries, people_ID to intake_Path
dtypes: datetime64[ns](20), float64(47), int64(3), object(187)
memory usage: 3.0+ MB


In [12]:
# add in dda information to the df

ddacase = pd.read_excel(r'redacted file path\DDA Case info - unformatted.xlsx')
ddacase = ddacase.add_prefix('dda_')

merge5 = merge4.merge(ddacase,how='left',left_on='intake_Client',right_on='dda_Individual')

In [13]:
# remove duplicates

merge5 = merge5.drop_duplicates(subset=['people_First Name','people_Last Name'])

In [14]:
# remove any text (within parethesis) to standardize names

merge5['people_First Name'] = merge5['people_First Name'].replace(r'\(.*?\)','',regex=True).str.strip()
merge5['people_Last Name'] = merge5['people_Last Name'].replace(r'\(.*?\)','',regex=True).str.strip()

merge5['people_First Name'] = merge5['people_First Name'].replace(r'\".*?\"','',regex=True).str.strip()
merge5['people_Last Name'] = merge5['people_Last Name'].replace(r'\".*?\"','',regex=True).str.strip()

In [15]:
merge6 = merge5[~merge5['people_Full Name'].str.contains('test', case=False, na=False)].copy()

In [19]:
from datetime import datetime

In [20]:
merge6 = merge6[merge6['people_Modified'] >= datetime(2023,1,1)].copy()


In [307]:
merge6.to_excel(r'redacted file path\exports\real people_merge2.xlsx')

#### Merge6 was changed to add in additional people

In [21]:
# added missing people to the merge6

merge6 = pd.read_excel(r'redacted file path\exports\real people_merge2.xlsx')

#### Individual profile start

In [22]:
merge6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Columns: 276 entries, people_ID to dda_Path
dtypes: bool(1), datetime64[ns](22), float64(98), int64(6), object(149)
memory usage: 1.1+ MB


In [23]:
# create a pseudo individual profiles import sheet

individual = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'intake_Nickname',
    'intake_Phone',
    'intake_Email',
    'intake_Address',
    'intake_City',
    'intake_State',
    'intake_Zip Code:County',
    'intake_Zip Code',
    'intake_Guardian:',
    'intake_Guardian: email',
    'intake_Guardian: First Name',
    'intake_Guardian: Last Name',
    'intake_Guardian: Mobile Phone',
    'intake_Guardian: Phone',
    'intake_Guardian: Status',
    'dda_DDA Case File Number',
    'dda_Funding Code',
    'dda_Acuity Level',
    'intake_Orion Location'
]]

In [24]:
# format the individual profile 
individual = individual.copy()
individual['Guardian info'] = individual['intake_Guardian: email']+' '+individual['intake_Guardian: First Name']+' '+individual['intake_Guardian: Last Name']+' '+individual['intake_Guardian: Phone']+' '+individual['intake_Guardian: Mobile Phone']+' '+individual['intake_Guardian: Status']

individual2 = individual.drop(columns=['intake_Guardian: email','intake_Guardian: First Name','intake_Guardian: Last Name','intake_Guardian: Phone','intake_Guardian: Mobile Phone','intake_Guardian: Status']).copy()
individual2['intake_Orion Location'] = individual2['intake_Orion Location'].replace('Auburn',1174)
individual2['intake_Orion Location'] = individual2['intake_Orion Location'].replace('Mukilteo',2870)

individual2['migrationID'] = range(1,len(individual) + 1)

In [25]:
# from datetime import datetime

individual2['demo_Date of Birth'] = individual2['demo_Date of Birth'].fillna('1/1/1800')

In [393]:
individual2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   people_First Name         528 non-null    object 
 1   people_Last Name          528 non-null    object 
 2   demo_Date of Birth        528 non-null    object 
 3   intake_Nickname           75 non-null     object 
 4   intake_Phone              438 non-null    object 
 5   intake_Email              389 non-null    object 
 6   intake_Address            438 non-null    object 
 7   intake_City               444 non-null    object 
 8   intake_State              456 non-null    object 
 9   intake_Zip Code:County    417 non-null    object 
 10  intake_Zip Code           428 non-null    float64
 11  intake_Guardian:          456 non-null    float64
 12  dda_DDA Case File Number  34 non-null     float64
 13  dda_Funding Code          32 non-null     object 
 14  dda_Acuity

In [394]:
individual2.to_excel(r'redacted file path\exports\real individual profile2.xlsx')

#### individual done

#### demographics start

In [26]:
demo = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'prof_Created',
    'demo_Gender',
    'demo_Ethnicity',
    'demo_Language Primary',
    'demo_Language Spoken',
    'demo_Marital Status',
    'demo_Veteran Status',
    'demo_Head Of Household',
    'demo_Living Situation',
    'demo_Immigrant and Refugee Status',
    'intake_Orion Location'
]].copy()

demo['intake_Orion Location'] = demo['intake_Orion Location'].replace('Auburn',1174)
demo['intake_Orion Location'] = demo['intake_Orion Location'].replace('Mukilteo',2870)

In [27]:
demo['demo_Gender'].value_counts()

demo_Gender
Male       335
Female     171
Trans        6
Other        2
Unknown      1
Name: count, dtype: int64

In [28]:
demo.loc[:,'demo_Gender'] = demo['demo_Gender'].str.replace('Trans','Transgender')

In [29]:
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].replace(r';#\d{2}','|',regex=True)
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].str.rstrip('|')
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].str.replace(';#','')

In [30]:
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].str.replace('Hawaiin','Hawaiian')
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].str.replace('African American / Black','Black / African American')
demo.loc[:,'demo_Ethnicity'] = demo['demo_Ethnicity'].str.replace('Multi-Racial','Other')

In [31]:
demo['demo_Language Primary'] = demo['demo_Language Primary'].str.replace('Other (please specify)','Other')

In [32]:
mar_rep = {'Divorced':'Single','Separated':'Single','Widowed':'Single'}

demo['demo_Marital Status'] = demo['demo_Marital Status'].replace(mar_rep)

In [33]:
vet_rep = {
    'Current Active Duty, Guard, Reserves or Veteran':'Currently AD, Guard, Reserves or Veteran',
    'Veteran - Present or Past ':'Currently AD, Guard, Reserves or Veteran',
    'Partner or Spouse of Active Service Member':'Partner or Spouse of Active Service Member/Veteran',
    'Minor Dependent of Active Service Member':'None'
}

demo['demo_Veteran Status'] = demo['demo_Veteran Status'].replace(vet_rep)
demo['demo_Veteran Status'] = demo['demo_Veteran Status'].fillna('None')

In [34]:
head_rep = {
    False:'No',
    True:'Yes'
}

demo['demo_Head Of Household'] = demo['demo_Head Of Household'].replace(head_rep)
demo['demo_Head Of Household'] = demo['demo_Head Of Household'].fillna('Unknown')

In [35]:
im_rep = {
    'Immigrant':'Immigrant status',
    'Refugee':'Refugee Status',
    'Unknown;#None':'Neither'
}

demo['demo_Immigrant and Refugee Status'] = demo['demo_Immigrant and Refugee Status'].replace(im_rep)
demo['demo_Immigrant and Refugee Status'] = demo['demo_Immigrant and Refugee Status'].fillna('Neither')

In [36]:
liverep = {
    'Homeless':'Homeless / Unhoused',
    'Independently / with Spouse or Partner':'Alone or with Spouse / Partner',
    'Other permanent housing (CSBG data)':'Other',
    'Own (CSBG data)':'Alone or with Spouse / Partner',
    'Rent (CSBG data)':'Alone or with Spouse / Partner',
    'Transitional / Temporary Housing':'Temporary/Transitional Housing',
    'With Parents':'With Parents or other/extended family',
    'With Extended Family':'With Parents or other/extended family'
}

demo['demo_Living Situation'] = demo['demo_Living Situation'].replace(liverep)
demo['demo_Living Situation'] = demo['demo_Living Situation'].fillna('Unknown')

In [37]:
demo['demo_Head Of Household'].value_counts()

demo_Head Of Household
0.0        479
1.0         39
Unknown     10
Name: count, dtype: int64

In [38]:
headrep = {
    0:'No',
    1:'Yes'
}

demo['demo_Head Of Household'] = demo['demo_Head Of Household'].replace(headrep)

In [39]:
demo['migrationID'] = range(1,len(demo) + 1)

In [409]:
demo.to_excel(r'redacted file path\exports\real demographics2.xlsx')

#### Demographics done

#### Intake Assessment start

In [41]:
for row in merge6:
    print(row)

people_ID
people_First Name
people_Full Name
people_Title
people_Forms Dropdowns
people_Created
people_Created By
people_Content Type
people_E-Mail
people_E-mail 2
people_Phone
people_Mobile Number
people_Professional Type
people_Comments
people_Filter
people_Folder Child Count
people_Item Child Count
people_Last Name
people_Modified
people_Modified By
people_Item Type
people_Path
demo_ID
demo_Individual
demo_Preferred Names
demo_Date of Birth
demo_Gender
demo_Disability - Primary
demo_Disabilities - Additional
demo_Disability / Barrier Notes:
demo_Date of Last Employment
demo_Smoker?
demo_Veteran - Discharge Date and Disposition
demo_If homeless, for how long?
demo_If homeless, last city of residence
demo_Emergency Contact Name
demo_Emergency Contact Number
demo_Emergency Contact Relationship
demo_Household Size
demo_Household Type
demo_Health Insurance
demo_Aboriginal Status
demo_Agency Exit Date
demo_Agency Intake Date
demo_Allergies
demo_Alt. Telephone Number
demo_Benefits
demo_Bla

In [43]:
intassess = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'demo_Education Level',
    'demo_Special Education History',
    'demo_Transportation',
    'demo_Allergies',
    'demo_Disability - Primary',
    'demo_Disabilities - Additional',
    'demo_Medications and Conditions',
    'demo_Criminal History',
    'demo_Smoker?',
    'demo_Household Income',
    'demo_Benefits',
    'demo_Health Insurance',
    'intake_Orion Location'
]].copy()

intassess['intake_Orion Location'] = intassess['intake_Orion Location'].replace('Auburn',1174)
intassess['intake_Orion Location'] = intassess['intake_Orion Location'].replace('Mukilteo',2870)

steps to do:

- yes/no for allergies
- yes/no for each mental disability
- yes/no for criminal history
- yes/no for household income

In [44]:
# create a column of disabilities and combine them together

intassess['disabilities'] = intassess['demo_Disability - Primary'] +' '+ intassess['demo_Disabilities - Additional']
intassess['disabilities'] = intassess['disabilities'].astype(str)

In [45]:
# add in a column for yes/no if they have a disability

mentallist = ['Depression','Anxiety','PTSD','ADHD','Substance Abuse','Learning Disability']

for i in mentallist:
    intassess[i] = intassess['disabilities'].apply(lambda x: 'Yes' if i in x else 'No')

In [46]:
edurep = {
    'No High School Diploma / GED':'No HS Diploma - Under 12th grade education',
    'In High School':'HS Diploma',
    'High School Diploma':'HS Diploma',
    'Graduated - In High School':'HS Diploma',
    'Some College / Technical School (no certificate / diploma)':'Some post high school',
    'Masters (or beyond)':'Masters and above',
    'Certificate':'Certificate - 2 years or less'
}

intassess['demo_Education Level'] = intassess['demo_Education Level'].replace(edurep)

In [47]:
intassess['demo_Transportation'].value_counts()

demo_Transportation
Car               208
Public Transit    171
Para-transit       39
Ride               28
Other              14
School Bus         10
Unknown             4
Bike                3
Name: count, dtype: int64

In [48]:
transrep = {
    'Car':'Driving Myself',
    'Public Transit':'Public Transportation',
    'Para-transit':'Paratransit',
    'Unknown':'Other'
}

intassess['demo_Transportation'] = intassess['demo_Transportation'].replace(transrep)

#### Intake Assessment done

### Start of people index 

In [49]:
peeps = pd.read_csv(r'redacted file path\for import\IndividualProfile import1 reviewed.csv')
peeps = peeps.drop(index=0)
peeps2 = peeps[['field_3436_first','field_3436_middle','field_3436_last','field_3195']].copy()
sep=','
peeps2['Individual'] = peeps['field_3436_last'].astype(str)+ sep + peeps['field_3436_first'].astype(str)

In [50]:
peeps3 = peeps2.drop_duplicates(subset=['field_3436_first','field_3436_middle','field_3436_last','field_3195'])

### Evaluation Merge

In [51]:
eval_list = ['eval - clampblock - unformatted', 
             'eval - custodial - unformatted', 
             'eval - finish - unformatted',  
             'eval - front desk - unformatted',
             'eval - grommets - unformatted',
             'eval - HBU - unformatted',
             'eval - machining cell - unformatted',
             'eval - Manufacturing office - unformatted',
             'eval - mechanical assembly - unformatted',
             'eval - mukilteo assembly - unformatted',
             'eval - mukilteo shipping and recieving - unformatted',
             'eval - mukilteo wires - unformatted',
             # 'eval - office skills - unformatted',
             'eval - QA - unformatted',
             'eval - sheet metal- unformatted',
             'eval - shipping and recieving - unformatted'#,
             # 'eval - soft skills - unformatted'
]


In [52]:
for i in eval_list:
    # establish the file path and file name
    path = 'redacted file path/'+i+'.xlsx'
    
    # set the dataframe to a list to access later
    df = pd.read_excel(path)

    if 'EvaluationSoftSkillsID' in df.columns:
        print(i+' has ID')
    else:
        print(i+' no ID')

eval - clampblock - unformatted has ID
eval - custodial - unformatted has ID
eval - finish - unformatted has ID
eval - front desk - unformatted has ID
eval - grommets - unformatted has ID
eval - HBU - unformatted has ID
eval - machining cell - unformatted has ID
eval - Manufacturing office - unformatted has ID
eval - mechanical assembly - unformatted has ID
eval - mukilteo assembly - unformatted has ID
eval - mukilteo shipping and recieving - unformatted has ID
eval - mukilteo wires - unformatted has ID
eval - QA - unformatted has ID
eval - sheet metal- unformatted has ID
eval - shipping and recieving - unformatted has ID


In [53]:
# set all evals to a list to iterat through later

eval_df = pd.DataFrame()

softskills = pd.read_excel('redacted file path/eval - soft skills - unformatted.xlsx')
softskills = softskills.add_suffix('_soft')

for i in eval_list:
    
    # establish the file path and file name
    path = 'redacted file path/'+i+'.xlsx'
    
    # set the dataframe to a list to access later
    df = pd.read_excel(path)

    # remove repetitive columns
    df = df.drop(columns=['Title','Item Type','Path','Folder Child Count','Created By','Content Type',])
    
    # remove the (###) from the individual name
    df.loc[:,'Individual'] = df['Individual'].replace(r'\b\d{1,4};#','',regex=True)

    # remove test case rows
    df = df[~df['Individual'].str.contains('test', case=False, na=False)].copy()

    # remove any text (within parethesis) to standardize names
    df.loc[:,'Individual'] = df['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

    df = df.merge(softskills,how='left',left_on=['EvaluationSoftSkillsID','Individual','Venue','Mentor'],right_on=['ID_soft','Individual_soft','Venue_soft','Mentor_soft'])
    
    eval_df = pd.concat([eval_df,df], ignore_index=True)

office = pd.read_excel('redacted file path/eval - office skills - unformatted.xlsx')
eval_df = pd.concat([eval_df,office])

In [54]:
eval_df2 = eval_df.copy()
eval_df2 = eval_df2.reset_index()

In [55]:
# remove space on the individual to allow for merging
eval_df2.loc[:,'Individual'] = eval_df2['Individual'].str.replace(', ',',')

# remove test case rows
eval_df2 = eval_df2[~eval_df2['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
eval_df2['Individual'] = eval_df2['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

eval_df2['Individual'] = eval_df2['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

eval_df2['Final Due Date'] = eval_df2['Due Date_soft'].combine_first(eval_df2['Evaluation Due Date'])
eval_df2 = eval_df2.drop(columns=['Evaluation Due Date','Due Date_soft'])

In [56]:
# change evaluation text to standardized text
eval_df2 = eval_df2.replace('Exemplary','Proficient',regex=True)
eval_df2 = eval_df2.replace('Unsatisfactory','Not Competitive',regex=True)
eval_df2 = eval_df2.replace('Inconsistent','Not Competitive',regex=True)
eval_df2 = eval_df2.replace('Did Not Train N/A','N/A',regex=True)
eval_df2 = eval_df2.replace('Did Not Train (N/A)','N/A',regex=False)

In [57]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [58]:
eval_df2 = eval_df2.merge(peepsmerge, how='right',left_on='Individual',right_on='people_Full Name')
eval_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5639 entries, 0 to 5638
Columns: 1014 entries, index to people_Full Name
dtypes: datetime64[ns](9), float64(25), object(980)
memory usage: 43.6+ MB


In [220]:
eval_df2.to_excel(r'redacted file path\exports\eval check1.xlsx')

In [59]:
eval_df2['location code'] = np.where(eval_df2['Venue'].str.contains('Auburn'),1174,2870)

In [60]:
eval_df3 = eval_df2.copy()

In [61]:
eval_df3['Work Cell'] = np.where(eval_df3['Venue'].str.contains('Auburn'),'Auburn '+eval_df3['Work Cell'],'Mukilteo '+eval_df3['Work Cell'])

In [62]:
# change work cell names to match apricot names

eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Clamp Block','Auburn Clamp Block Issuing')
eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Finish','Auburn Racking / Unracking; Paint Prep')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Sheet Metal','')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Shipping and Recieving','Auburn Shipping')
eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Mukilteo Wire','Mukilteo Wire Assembly')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Machining','')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Office (Manufacturing)','')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Mukilteo Assembly','')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Contact Center','Auburn Clerical Customer Service')
eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Mukilteo Shipping and Receiving','Mukilteo Shipping & Receiving')
eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Auburn Quality Assurance','Auburn QA')
### eval_df3['Work Cell'] = eval_df3['Work Cell'].str.replace('Mukilteo Machining','')

eval_df3['Work Cell'].value_counts()

Work Cell
Auburn Clamp Block Issuing                1423
Auburn Sheet Metal                         673
Mukilteo Wire Assembly                     575
Auburn Racking / Unracking; Paint Prep     563
Auburn Mills                               475
Auburn Mechanical Assembly                 396
Auburn Shipping and Receiving              247
Auburn Machining                           211
Mukilteo Assembly                          204
Mukilteo Shipping & Receiving              135
Auburn Office (Manufacturing)              122
Mukilteo Mechanical Assembly               116
Auburn HBU                                 108
Auburn Custodial                            65
Auburn Contact Center                       56
Auburn QA                                   29
Mukilteo Machining                           8
Auburn Assembly                              2
Name: count, dtype: int64

In [226]:
eval_df3.to_excel(r'redacted file path\exports\eval merge 5.xlsx')

In [229]:
for i in eval_df3.columns:
    print(i)

index
Venue
Mentor
Individual
Work Cell
Work Center
Grommets:
Modified
Packaging & Part Mark:
Assembly:
Assembly: Accurately faces the blocks the same way in the box
Assembly: Accurately measures screws and spacers using calipers
Assembly: Accurately verifies parts with paperwork
Assembly: Comments for Basic Manufacturing Task List SectionFilter
Assembly: Comments on Advanced Manufacturing Tasks List SectionFilter
Assembly: Demonstrates ability to locate correct part when pulling standards
Assembly: Demonstrates ability to place jobs on correct spot of warehouse shelf
Assembly: Demonstrating the ability to properly restock inventory to correct area
Assembly: Follows posted assembly instructions accurately
Assembly: Independent ability to accurately read the details of a blueprint
Assembly: Independent ability to build a clamp block correctly
Assembly: Independent ability to locate blueprint for the job
Assembly: Properly lays out the blocks for building
Assembly: Replacing paperwork in

In [63]:
eval_df3 = eval_df3.replace(';#','|',regex=True)

In [64]:
evaldf = eval_df3[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Mentor',
    'Work Cell',
    'For the Week of:_soft',
    'Final Due Date',
    'Attendance (Please note number of full days missed. If no days were missed, fill in 0)_soft',
    'Punctuality_soft',
    'Demeanor & Attitude_soft',
    'Workplace Appearance & Attire_soft',
    'Grooming & Hygiene_soft',
    'Cooperation & Teamwork_soft',
    'Interpersonal Communication Style_soft',
    'Response to Feedback / Instruction_soft',
    'Organization_soft',
    'Work Effort & Initiative_soft',
    'Work Tolerance & Stamina_soft',
    'Learning Style_soft',
    'Follows all Safety Rules & Policies?_soft',
    'Note Any Inappropriate Behaviors Observed during Evaluation Period (if no inappropriate behaviors observed, mark that choice)_soft',
    "Are Person's Soft Skills / General Employment Traits Competitive?_soft",
    'Soft Skills Comments_soft',
    # general employment
    'General Employment: Keeps a clean & organized workspace in order to reduce FOD',
    # Utilizes pictures/blueprint to produce clamp block assembly
    'Assembly: Independent ability to accurately read the details of a blueprint',
    # Lays out blocks for building correctly
    'Assembly: Properly lays out the blocks for building',
    # Places jobs on correct location on shelves|Returns excess pieces to inventory
    'Assembly: Demonstrates ability to place jobs on correct spot of warehouse shelf',
    #Sets up for part mark, including part mark location and selecting proper size for part mark
    'Packaging & Part Mark: Proper set up for part mark',
    #Assesses part mark for quality and redo as needed|Applies overcoat
    'Packaging & Part Mark: Applies overcoat precisely, with desired thickness',
    #Judges size of bag to package parts|Packages multiple parts that will meet inspection standards|Operates bag sealer
    'Packaging & Part Mark: Correct operation of bag sealer',
    #Removes labels from boxes before use; Creates labels for job boxes
    'Clamp Block Inventory: Creates labels for job boxes',
    #When pulling parts from inventory - Identifies and verifies part #'s and lot #'s using the Job Material Queue
    'Clamp Block Inventory: For each part pulled from inventory, records correct lot # and qty to work order',
    #Issues parts with computer using Job Material Queue and Process by ID
    'Clamp Block Inventory: Issues parts with computer using Inventory Transaction in VMFG',
    #Operates the chop saw
    'CHAKU: Operation of saw',
    #Operates the punch press
    'CHAKU: Operation of punch press',
    #Operates the countersink machine
    'CHAKU: Operation of countersink',
    #Operates the flat finisher (deburr)
    'CHAKU: Operation of deburr',
    #Operates the CNC Mill
    'CHAKU: Operation of CNC Mill',
    #Utilizes work order to differentiate between grommet shape: (ex. F material from J or H material)
    'Grommets: Utilizes work order to differentiate between grommet shape',
    #Kiss cuts washers
    'Grommets: Demonstrates ability to kiss cut washers',
    #Wraps prepped grommet onto centers; loads into jig
    'Grommets: Independent ability to accurately load a jig',
    #Assembles a grommet|Identifies holes in grommet|Repairs holes in grommet|Builds at average rate of 10-15 parts per hour
    'Grommets: Ability to identify holes in grommet',
    #Cleans offices, corridors, meeting rooms, and common spaces|Dusts|Cleans restrooms|Uses auto scrubber to clean floor|Vacuums carpeted floors|Washes windows and doors|Takes out trash and recycling
    'Custodial: Follows proper restroom cleaning sequence that meets quality standards',
    #Finds SDS information|Works with chemicals and mixes cleaning solutions
    'Custodial: Independent ability to properly mix cleaning solutions',
    #Checks appliances and light fixtures, changes light bulbs as needed
    'Custodial: Independently changes light bulbs as needed',
    #Identifies each job and work order number|Identifies the process for each part/job|Creates identification tags for jobs travelling through processes
    'Racking / Unracking; Paint Prep: Independent ability to obtain process information from work order traveler',
    #Masks parts using tape and/or plugs|Removes masking tape and/or plugs|Lays out parts on paint screen
    'Racking / Unracking; Paint Prep: Able to properly layout parts on paint screen',
    #Pre-cleans parts by removing ink or excessive oils
    'Racking / Unracking; Paint Prep: Pre-cleans parts by removing ink or excessive oils',
    #Enters part mark data on the A200 or A320i part mark machine|Places part mark in correct location
    'Part Mark: Independently utilize the A320i vendor options for data entry of part mark',
    #Uploads part mark onto a dauber sheet|Applies part mark using ink dauber
    'Part Mark: Application of part marks by using ink dauber',
    #Ability to correctly judge what size bag to package parts in
    'Packaging: Ability to correctly judge what size bag to package parts in',
    #Greets visitors and assists with purpose of visit|Answers telephone and direct calls|Handles customer questions or issues requiring follow up
    'General Admin Clerk: Provides excellent customer service',
    #Checks daily mail and delivers
    'General Admin Clerk: Independent with the mail run (grabs mail from mailbox, sorts and delivers to appropriate parties)',
    #Scans documents and saves files to the appropriate location on computer|Uses photocopy machine|Creates paperwork packets
    'General Admin Clerk: Saving documents/files in the appropriate location on the computer',
    #Completes over coat operations
    'HBU: Independently completes over coating operations',
    #Completes job shadow of an assembly technician
    'HBU: Completion of job shadow of an assembly technician',
    #Identifies call outs, flags notes and locations for parts on blueprints
    'HBU - assembly: Recognizes views and orientation of parts on blueprints',
    #Builds parts while being shadowed by mentor/technician
    'HBU - assembly: Accurately builds parts while being shadowed by mentor/technician',
    #Receives certifications (note in comments which certifications)
    'HBU - assembly: Receives certifications (note in comments which certifications)',
    #Loads and unloads tooling from holders and spindle
    'Mills: Independent ability to load and unload tools from holders and spindle',
    #Identifies burrs|Deburrs parts
    'Mills: Independent ability to deburr parts correctly',
    #Understands and explains concept of X, Y, Z coordinate system
    'Mills: Able to understand and explain concept of X, Y, Z coordinate system',
    #Sets up tools|Breaks down returned tools
    'Mills Tool Crib: Accurately sets up tools',
    #Identifies burrs, damage and flaws
    'Vibraburr: Independent ability to identify a burr',
    #Runs Vibrodyne machine|Runs Roto machine|Runs Burr King machine|Runs Cement Mixer machine
    'Vibraburr: Independent ability to run Vibrodyne machine',
    #Identifies when machine cleaning is needed (ex. change water)
    'Vibraburr: Independent ability to clean machine as needed (ex. change water)',
    #Identifies and stamps military export controlled work orders|Identifies work orders with FAI (First Article Inspection) operation|Sorts, staples, and confirms documents are in order
    'Commercial Work Order Processing: IDs and stamps military export controlled work orders',
    #Verifies that revision levels listed on work order are current (sources of reference may include Boeing EID and customer purchase order notes)
    'Commercial Work Order Processing: Verifies revision levels listed on work order are current',
    #Opens estimating tab in Epicor and inputs data into correct fields|Opens and reviews estimating quote log in Epicor|Logs onto Exostar/Boeing portal|Opens EID (Engineering Information Delivery) page, selects & downloads correct drawings on Boeing Portal
    'Estimating Training Evaluation: Correctly opens EID page, selects & downloads correct drawings on Boeing Portal',
    #Identifies drill bits and countersink size
    'Mechanical Assembly: Independent, accurate usage of precision measuring equipment',
    #Operates the countersink machine|Sets up the countersink machine
    'Mechanical Assembly: Independent operation of countersink',
    #Operates the riveter/squeeze machine|Sets up the riveter/squeeze machine
    'Mechanical Assembly: Independent operation of riveter / squeeze',
    #Operates the drill press|Sets up the drill press
    'Mechanical Assembly: Independent operation of drill press',
    #Utilizes procedure for bonding tasks
    'Mechanical Assembly : Utilizes proper procedure for bonding tasks',
    #Assembles wiggle plates
    'Mechanical Assembly : Assembly of wiggle plates',
    #Sets up minor jobs (nut plate, wiggle plate, countersink machine)
    'Mechanical Assembly : Independent set-up of minor jobs (nut plate, wiggle plate, countersink machine)',
    #Determines drawings required
    'QA: Ability to determine drawings required and current revisions',
    #Operates the blanking press
    'Blanking Press: Acceptable operation of blanking press',
    #Locates tooling for each job|Sets up and takes down tooling|Puts tool away
    'Blanking Press: Ability to locate proper tooling for each job',
    #Identifies burrs|Deburrs parts
    'Blanking Press: Correctly deburrs parts',
    #Lubricates tools
    'Blanking Press: Proper lubrication of material',
    #Operates the Amada
    'Amada: Proper operation of Amada',
    #Operates brake form in Mylar cutting
    'Brake Form: Operation of brake form in Mylar cutting',
    #Sets up and centers Mylar cutting tool
    'Brake Form: Independent ability to set up and center Mylar cutting tool',
    #Identifies burrs|Uses "Timesaver"|Deburrs parts
    'Brake Form: Correctly deburrs parts',
    #Identifies and locates correct material for each job|Loads and unloads material|Performs computer steps to start machine
    'Routers: Independent ability to identify and locate correct material for each job',
    #Identifies and locates tooling for each job|Sets up and changes tooling plates
    'Routers: Independent ability to setup and change tooling plates',
    #Identifies burrs|Deburrs parts
    'Routers: Correctly deburrs parts',
    #Differentiates between Boeing and Commercial work orders|Locates tools and templates used for Boeing work orders|Locates, prints, and verifies certification (cert)
    'Shears: Independent ability to differentiate between Boeing and Commercial work orders',
    #Sets up the machine
    'Shears: Independent ability to set up machine',
    #Operates the shear
    'Shears: Accurate operation of Shear',
    #Understands machine operation and computer inputs|Differentiates between Automatic Start and Manual Start procedure and when each would be used
    'Waterjet: Understands and demonstrates proper machine operation and computer inputs',
    #Operates the water jet|Sets up the water jet
    'Waterjet: Independent setup and machine operation of Water Jet',
    #Operates the grind wheel|Uses deburr hand tools including scotch brite or sand paper|Cleans parts thoroughly with alcohol
    'Deburr: Cleans parts thoroughly with alcohol',
    #Counts and verifies part mark and paperwork prior to packaging outgoing shipment
    'Commercial Shipping: Sort, segregate and prioritize verified work order by due date and shipping date',
    #Visually inspects incoming product for damage, re-package product according to Orion guidelines prior to moving on to next area
    'Receiving: Independence with receiving materials and performing tasks in Receiving work area',
    #Operates the pallet jack
    'Receiving: Independent utilization of pallet jack',
    #Visually inspects incoming product for damage, re-packages product per Orion guidelines prior to moving into inventory
    "Inventory: for parts pulled out of inventory, verifies correct part #'s on work order",
    #Retrieves product from inventory; moves to next location based on work order or pull ticket
    'Inventory: Independently finds correct location when picking parts from inventory',
    #Counts product in each specified bin location during cycle count activity
    'Inventory: Ensures that counts in inventory match what is posted for cycle counts',
    #Applies bond and adheres waffle or strips to orifice|Places weights on waffle orifices
    'Bond: Application of bond and adherence of waffle or strips to orifice',
    #Calculates cure time for bond
    'Bond: Independent, accurate calculation of cure time for bond',
    #Creates internal work order
    'Shipping & Receiving: Independent creation of internal work order',
    #Enters information into Boeing Portal
    'Shipping & Receiving: Accurate entering of information into Boeing Portal',
    #Utilizes and navigates computer applications
    'Manufacturing Offices: Independent ability to utilize and navigate computer applications',
    #Creates Purchase Orders|Creates quotes/estimates
    'Manufacturing Offices: Independently creates Purchase Orders',
    #Gathers data from internal lists and makes determinations based on information
    'Manufacturing Offices: Gathers correct data from internal lists and makes correct decisions based on this informaton',
    #Visually inspects and measures parts
    'Wire Assembly: Independent visual inspection and measurement of each part',
    #Ensures that counts per bag match what is posted on front of boxes|Properly runs 2 bag system
    'Inventory: Proper run of 2 bag system',
    #Inspects and verifies wire part marks/lengths|Completes job (operations closed, paperwork stamped, etc.)
    'Wire QA: Accurate inspection and verification of wire lengths',
    #Loads and threads wire through de-reeler, unracks wire spool and returns to its correct location
    'Laser Marking: Independently and accurately loads and threads wire through de-reeler',
    #Follows procedure to access laser marking information on flash drive/computer
    'Laser Marking: Follows proper procedure to access laser marking information on flash drive/computer',
    'location code'
]]

In [65]:
evalreplist = ['Keeps a clean & organized workspace in order to reduce FOD',
'General Employment: Keeps a clean & organized workspace in order to reduce FOD',
'Utilizes pictures/blueprint to produce clamp block assembly',
'Assembly: Independent ability to accurately read the details of a blueprint',
'Lays out blocks for building correctly',
'Assembly: Properly lays out the blocks for building',
'Places jobs on correct location on shelves|Returns excess pieces to inventory',
'Assembly: Demonstrates ability to place jobs on correct spot of warehouse shelf',
'Sets up for part mark, including part mark location and selecting proper size for part mark',
'Packaging & Part Mark: Proper set up for part mark',
'Assesses part mark for quality and redo as needed|Applies overcoat',
'Packaging & Part Mark: Applies overcoat precisely, with desired thickness',
'Judges size of bag to package parts|Packages multiple parts that will meet inspection standards|Operates bag sealer',
'Packaging & Part Mark: Correct operation of bag sealer',
'Removes labels from boxes before use; Creates labels for job boxes',
'Clamp Block Inventory: Creates labels for job boxes',
"When pulling parts from inventory - Identifies and verifies part #'s and lot #'s using the Job Material Queue",
'Clamp Block Inventory: For each part pulled from inventory, records correct lot # and qty to work order',
'Issues parts with computer using Job Material Queue and Process by ID',
'Clamp Block Inventory: Issues parts with computer using Inventory Transaction in VMFG',
'Operates the chop saw',
'CHAKU: Operation of saw',
'Operates the punch press',
'CHAKU: Operation of punch press',
'Operates the countersink machine',
'CHAKU: Operation of countersink',
'Operates the flat finisher (deburr)',
'CHAKU: Operation of deburr',
'Operates the CNC Mill',
'CHAKU: Operation of CNC Mill',
'Utilizes work order to differentiate between grommet shape: (ex. F material from J or H material)',
'Grommets: Utilizes work order to differentiate between grommet shape',
'Kiss cuts washers',
'Grommets: Demonstrates ability to kiss cut washers',
'Wraps prepped grommet onto centers; loads into jig',
'Grommets: Independent ability to accurately load a jig',
'Assembles a grommet|Identifies holes in grommet|Repairs holes in grommet|Builds at average rate of 10-15 parts per hour',
'Grommets: Ability to identify holes in grommet',
'Cleans offices, corridors, meeting rooms, and common spaces|Dusts|Cleans restrooms|Uses auto scrubber to clean floor|Vacuums carpeted floors|Washes windows and doors|Takes out trash and recycling',
'Custodial: Follows proper restroom cleaning sequence that meets quality standards',
'Finds SDS information|Works with chemicals and mixes cleaning solutions',
'Custodial: Independent ability to properly mix cleaning solutions',
'Checks appliances and light fixtures, changes light bulbs as needed',
'Custodial: Independently changes light bulbs as needed',
'Identifies each job and work order number|Identifies the process for each part/job|Creates identification tags for jobs travelling through processes',
'Racking / Unracking; Paint Prep: Independent ability to obtain process information from work order traveler',
'Masks parts using tape and/or plugs|Removes masking tape and/or plugs|Lays out parts on paint screen',
'Racking / Unracking; Paint Prep: Able to properly layout parts on paint screen',
'Pre-cleans parts by removing ink or excessive oils',
'Racking / Unracking; Paint Prep: Pre-cleans parts by removing ink or excessive oils',
'Enters part mark data on the A200 or A320i part mark machine|Places part mark in correct location',
'Part Mark: Independently utilize the A320i vendor options for data entry of part mark',
'Uploads part mark onto a dauber sheet|Applies part mark using ink dauber',
'Part Mark: Application of part marks by using ink dauber',
'Ability to correctly judge what size bag to package parts in',
'Packaging: Ability to correctly judge what size bag to package parts in',
'Greets visitors and assists with purpose of visit|Answers telephone and direct calls|Handles customer questions or issues requiring follow up',
'General Admin Clerk: Provides excellent customer service',
'Checks daily mail and delivers',
'General Admin Clerk: Independent with the mail run (grabs mail from mailbox, sorts and delivers to appropriate parties)',
'Scans documents and saves files to the appropriate location on computer|Uses photocopy machine|Creates paperwork packets',
'General Admin Clerk: Saving documents/files in the appropriate location on the computer',
'Completes over coat operations',
'HBU: Independently completes over coating operations',
'Completes job shadow of an assembly technician',
'HBU: Completion of job shadow of an assembly technician',
'Identifies call outs, flags notes and locations for parts on blueprints',
'HBU - assembly: Recognizes views and orientation of parts on blueprints',
'Builds parts while being shadowed by mentor/technician',
'HBU - assembly: Accurately builds parts while being shadowed by mentor/technician',
'Receives certifications (note in comments which certifications)',
'HBU - assembly: Receives certifications (note in comments which certifications)',
'Loads and unloads tooling from holders and spindle',
'Mills: Independent ability to load and unload tools from holders and spindle',
'Identifies burrs|Deburrs parts',
'Mills: Independent ability to deburr parts correctly',
'Understands and explains concept of X, Y, Z coordinate system',
'Mills: Able to understand and explain concept of X, Y, Z coordinate system',
'Sets up tools|Breaks down returned tools',
'Mills Tool Crib: Accurately sets up tools',
'Identifies burrs, damage and flaws',
'Vibraburr: Independent ability to identify a burr',
'Runs Vibrodyne machine|Runs Roto machine|Runs Burr King machine|Runs Cement Mixer machine',
'Vibraburr: Independent ability to run Vibrodyne machine',
'Identifies when machine cleaning is needed (ex. change water)',
'Vibraburr: Independent ability to clean machine as needed (ex. change water)',
'Identifies and stamps military export controlled work orders|Identifies work orders with FAI (First Article Inspection) operation|Sorts, staples, and confirms documents are in order',
'Commercial Work Order Processing: IDs and stamps military export controlled work orders',
'Verifies that revision levels listed on work order are current (sources of reference may include Boeing EID and customer purchase order notes)',
'Commercial Work Order Processing: Verifies revision levels listed on work order are current',
'Opens estimating tab in Epicor and inputs data into correct fields|Opens and reviews estimating quote log in Epicor|Logs onto Exostar/Boeing portal|Opens EID (Engineering Information Delivery) page, selects & downloads correct drawings on Boeing Portal',
'Estimating Training Evaluation: Correctly opens EID page, selects & downloads correct drawings on Boeing Portal',
'Identifies drill bits and countersink size',
'Mechanical Assembly: Independent, accurate usage of precision measuring equipment',
'Operates the countersink machine|Sets up the countersink machine',
'Mechanical Assembly: Independent operation of countersink',
'Operates the riveter/squeeze machine|Sets up the riveter/squeeze machine',
'Mechanical Assembly: Independent operation of riveter / squeeze',
'Operates the drill press|Sets up the drill press',
'Mechanical Assembly: Independent operation of drill press',
'Utilizes procedure for bonding tasks',
'Mechanical Assembly : Utilizes proper procedure for bonding tasks',
'Assembles wiggle plates',
'Mechanical Assembly : Assembly of wiggle plates',
'Sets up minor jobs (nut plate, wiggle plate, countersink machine)',
'Mechanical Assembly : Independent set-up of minor jobs (nut plate, wiggle plate, countersink machine)',
'Determines drawings required',
'QA: Ability to determine drawings required and current revisions',
'Operates the blanking press',
'Blanking Press: Acceptable operation of blanking press',
'Locates tooling for each job|Sets up and takes down tooling|Puts tool away',
'Blanking Press: Ability to locate proper tooling for each job',
'Identifies burrs|Deburrs parts',
'Blanking Press: Correctly deburrs parts',
'Lubricates tools',
'Blanking Press: Proper lubrication of material',
'Operates the Amada',
'Amada: Proper operation of Amada',
'Operates brake form in Mylar cutting',
'Brake Form: Operation of brake form in Mylar cutting',
'Sets up and centers Mylar cutting tool',
'Brake Form: Independent ability to set up and center Mylar cutting tool',
'Identifies burrs|Uses "Timesaver"|Deburrs parts',
'Brake Form: Correctly deburrs parts',
'Identifies and locates correct material for each job|Loads and unloads material|Performs computer steps to start machine',
'Routers: Independent ability to identify and locate correct material for each job',
'Identifies and locates tooling for each job|Sets up and changes tooling plates',
'Routers: Independent ability to setup and change tooling plates',
'Identifies burrs|Deburrs parts',
'Routers: Correctly deburrs parts',
'Differentiates between Boeing and Commercial work orders|Locates tools and templates used for Boeing work orders|Locates, prints, and verifies certification (cert)',
'Shears: Independent ability to differentiate between Boeing and Commercial work orders',
'Sets up the machine',
'Shears: Independent ability to set up machine',
'Operates the shear',
'Shears: Accurate operation of Shear',
'Understands machine operation and computer inputs|Differentiates between Automatic Start and Manual Start procedure and when each would be used',
'Waterjet: Understands and demonstrates proper machine operation and computer inputs',
'Operates the water jet|Sets up the water jet',
'Waterjet: Independent setup and machine operation of Water Jet',
'Operates the grind wheel|Uses deburr hand tools including scotch brite or sand paper|Cleans parts thoroughly with alcohol',
'Deburr: Cleans parts thoroughly with alcohol',
'Counts and verifies part mark and paperwork prior to packaging outgoing shipment',
'Commercial Shipping: Sort, segregate and prioritize verified work order by due date and shipping date',
'Visually inspects incoming product for damage, re-package product according to Orion guidelines prior to moving on to next area',
'Receiving: Independence with receiving materials and performing tasks in Receiving work area',
'Operates the pallet jack',
'Receiving: Independent utilization of pallet jack',
'Visually inspects incoming product for damage, re-packages product per Orion guidelines prior to moving into inventory',
"Inventory: for parts pulled out of inventory, verifies correct part #'s on work order",
'Retrieves product from inventory; moves to next location based on work order or pull ticket',
'Inventory: Independently finds correct location when picking parts from inventory',
'Counts product in each specified bin location during cycle count activity',
'Inventory: Ensures that counts in inventory match what is posted for cycle counts',
'Applies bond and adheres waffle or strips to orifice|Places weights on waffle orifices',
'Bond: Application of bond and adherence of waffle or strips to orifice',
'Calculates cure time for bond',
'Bond: Independent, accurate calculation of cure time for bond',
'Creates internal work order',
'Shipping & Receiving: Independent creation of internal work order',
'Enters information into Boeing Portal',
'Shipping & Receiving: Accurate entering of information into Boeing Portal',
'Utilizes and navigates computer applications',
'Manufacturing Offices: Independent ability to utilize and navigate computer applications',
'Creates Purchase Orders|Creates quotes/estimates',
'Manufacturing Offices: Independently creates Purchase Orders',
'Gathers data from internal lists and makes determinations based on information',
'Manufacturing Offices: Gathers correct data from internal lists and makes correct decisions based on this informaton',
'Visually inspects and measures parts',
'Wire Assembly: Independent visual inspection and measurement of each part',
'Ensures that counts per bag match what is posted on front of boxes|Properly runs 2 bag system',
'Inventory: Proper run of 2 bag system',
'Inspects and verifies wire part marks/lengths|Completes job (operations closed, paperwork stamped, etc.)',
'Wire QA: Accurate inspection and verification of wire lengths',
'Loads and threads wire through de-reeler, unracks wire spool and returns to its correct location',
'Laser Marking: Independently and accurately loads and threads wire through de-reeler',
'Follows procedure to access laser marking information on flash drive/computer',
'Laser Marking: Follows proper procedure to access laser marking information on flash drive/computer']

In [66]:
evaldf2 = evaldf.copy()

In [67]:
for i,v in enumerate(evalreplist):
    i1 = i+1
    try:
        v1 = evalreplist[i1]
    except:
        print('done')
    if i%2 == 0:
        evaldf2[v] = np.where(evaldf2[v1].notna(),v,'')
    else:
        continue

done


In [68]:
evaldf2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5639 entries, 0 to 5638
Columns: 200 entries, people_First Name to Follows procedure to access laser marking information on flash drive/computer
dtypes: datetime64[ns](2), float64(1), int32(1), object(196)
memory usage: 8.6+ MB


In [69]:
evaldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5639 entries, 0 to 5638
Columns: 113 entries, people_First Name to location code
dtypes: datetime64[ns](2), float64(1), int32(1), object(109)
memory usage: 4.8+ MB


In [70]:
evaldf4 = evaldf2[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Mentor',
    'Work Cell',
    'For the Week of:_soft',
    'Final Due Date',
    'Attendance (Please note number of full days missed. If no days were missed, fill in 0)_soft',
    'Punctuality_soft',
    'Demeanor & Attitude_soft',
    'Workplace Appearance & Attire_soft',
    'Grooming & Hygiene_soft',
    'Cooperation & Teamwork_soft',
    'Interpersonal Communication Style_soft',
    'Response to Feedback / Instruction_soft',
    'Organization_soft',
    'Work Effort & Initiative_soft',
    'Work Tolerance & Stamina_soft',
    'Learning Style_soft',
    'Follows all Safety Rules & Policies?_soft',
    'Note Any Inappropriate Behaviors Observed during Evaluation Period (if no inappropriate behaviors observed, mark that choice)_soft',
    "Are Person's Soft Skills / General Employment Traits Competitive?_soft",
    'Soft Skills Comments_soft',
    'Keeps a clean & organized workspace in order to reduce FOD',
    'General Employment: Keeps a clean & organized workspace in order to reduce FOD',
    'Utilizes pictures/blueprint to produce clamp block assembly',
    'Assembly: Independent ability to accurately read the details of a blueprint',
    'Lays out blocks for building correctly',
    'Assembly: Properly lays out the blocks for building',
    'Places jobs on correct location on shelves|Returns excess pieces to inventory',
    'Assembly: Demonstrates ability to place jobs on correct spot of warehouse shelf',
    'Sets up for part mark, including part mark location and selecting proper size for part mark',
    'Packaging & Part Mark: Proper set up for part mark',
    'Assesses part mark for quality and redo as needed|Applies overcoat',
    'Packaging & Part Mark: Applies overcoat precisely, with desired thickness',
    'Judges size of bag to package parts|Packages multiple parts that will meet inspection standards|Operates bag sealer',
    'Packaging & Part Mark: Correct operation of bag sealer',
    'Removes labels from boxes before use; Creates labels for job boxes',
    'Clamp Block Inventory: Creates labels for job boxes',
    "When pulling parts from inventory - Identifies and verifies part #'s and lot #'s using the Job Material Queue",
    'Clamp Block Inventory: For each part pulled from inventory, records correct lot # and qty to work order',
    'Issues parts with computer using Job Material Queue and Process by ID',
    'Clamp Block Inventory: Issues parts with computer using Inventory Transaction in VMFG',
    'Operates the chop saw',
    'CHAKU: Operation of saw',
    'Operates the punch press',
    'CHAKU: Operation of punch press',
    'Operates the countersink machine',
    'CHAKU: Operation of countersink',
    'Operates the flat finisher (deburr)',
    'CHAKU: Operation of deburr',
    'Operates the CNC Mill',
    'CHAKU: Operation of CNC Mill',
    'Utilizes work order to differentiate between grommet shape: (ex. F material from J or H material)',
    'Grommets: Utilizes work order to differentiate between grommet shape',
    'Kiss cuts washers',
    'Grommets: Demonstrates ability to kiss cut washers',
    'Wraps prepped grommet onto centers; loads into jig',
    'Grommets: Independent ability to accurately load a jig',
    'Assembles a grommet|Identifies holes in grommet|Repairs holes in grommet|Builds at average rate of 10-15 parts per hour',
    'Grommets: Ability to identify holes in grommet',
    'Cleans offices, corridors, meeting rooms, and common spaces|Dusts|Cleans restrooms|Uses auto scrubber to clean floor|Vacuums carpeted floors|Washes windows and doors|Takes out trash and recycling',
    'Custodial: Follows proper restroom cleaning sequence that meets quality standards',
    'Finds SDS information|Works with chemicals and mixes cleaning solutions',
    'Custodial: Independent ability to properly mix cleaning solutions',
    'Checks appliances and light fixtures, changes light bulbs as needed',
    'Custodial: Independently changes light bulbs as needed',
    'Identifies each job and work order number|Identifies the process for each part/job|Creates identification tags for jobs travelling through processes',
    'Racking / Unracking; Paint Prep: Independent ability to obtain process information from work order traveler',
    'Masks parts using tape and/or plugs|Removes masking tape and/or plugs|Lays out parts on paint screen',
    'Racking / Unracking; Paint Prep: Able to properly layout parts on paint screen',
    'Pre-cleans parts by removing ink or excessive oils',
    'Racking / Unracking; Paint Prep: Pre-cleans parts by removing ink or excessive oils',
    'Enters part mark data on the A200 or A320i part mark machine|Places part mark in correct location',
    'Part Mark: Independently utilize the A320i vendor options for data entry of part mark',
    'Uploads part mark onto a dauber sheet|Applies part mark using ink dauber',
    'Part Mark: Application of part marks by using ink dauber',
    'Ability to correctly judge what size bag to package parts in',
    'Packaging: Ability to correctly judge what size bag to package parts in',
    'Greets visitors and assists with purpose of visit|Answers telephone and direct calls|Handles customer questions or issues requiring follow up',
    'General Admin Clerk: Provides excellent customer service',
    'Checks daily mail and delivers',
    'General Admin Clerk: Independent with the mail run (grabs mail from mailbox, sorts and delivers to appropriate parties)',
    'Scans documents and saves files to the appropriate location on computer|Uses photocopy machine|Creates paperwork packets',
    'General Admin Clerk: Saving documents/files in the appropriate location on the computer',
    'Completes over coat operations',
    'HBU: Independently completes over coating operations',
    'Completes job shadow of an assembly technician',
    'HBU: Completion of job shadow of an assembly technician',
    'Identifies call outs, flags notes and locations for parts on blueprints',
    'HBU - assembly: Recognizes views and orientation of parts on blueprints',
    'Builds parts while being shadowed by mentor/technician',
    'HBU - assembly: Accurately builds parts while being shadowed by mentor/technician',
    'Receives certifications (note in comments which certifications)',
    'HBU - assembly: Receives certifications (note in comments which certifications)',
    'Loads and unloads tooling from holders and spindle',
    'Mills: Independent ability to load and unload tools from holders and spindle',
    'Identifies burrs|Deburrs parts',
    'Mills: Independent ability to deburr parts correctly',
    'Understands and explains concept of X, Y, Z coordinate system',
    'Mills: Able to understand and explain concept of X, Y, Z coordinate system',
    'Sets up tools|Breaks down returned tools',
    'Mills Tool Crib: Accurately sets up tools',
    'Identifies burrs, damage and flaws',
    'Vibraburr: Independent ability to identify a burr',
    'Runs Vibrodyne machine|Runs Roto machine|Runs Burr King machine|Runs Cement Mixer machine',
    'Vibraburr: Independent ability to run Vibrodyne machine',
    'Identifies when machine cleaning is needed (ex. change water)',
    'Vibraburr: Independent ability to clean machine as needed (ex. change water)',
    'Identifies and stamps military export controlled work orders|Identifies work orders with FAI (First Article Inspection) operation|Sorts, staples, and confirms documents are in order',
    'Commercial Work Order Processing: IDs and stamps military export controlled work orders',
    'Verifies that revision levels listed on work order are current (sources of reference may include Boeing EID and customer purchase order notes)',
    'Commercial Work Order Processing: Verifies revision levels listed on work order are current',
    'Opens estimating tab in Epicor and inputs data into correct fields|Opens and reviews estimating quote log in Epicor|Logs onto Exostar/Boeing portal|Opens EID (Engineering Information Delivery) page, selects & downloads correct drawings on Boeing Portal',
    'Estimating Training Evaluation: Correctly opens EID page, selects & downloads correct drawings on Boeing Portal',
    'Identifies drill bits and countersink size',
    'Mechanical Assembly: Independent, accurate usage of precision measuring equipment',
    'Operates the countersink machine|Sets up the countersink machine',
    'Mechanical Assembly: Independent operation of countersink',
    'Operates the riveter/squeeze machine|Sets up the riveter/squeeze machine',
    'Mechanical Assembly: Independent operation of riveter / squeeze',
    'Operates the drill press|Sets up the drill press',
    'Mechanical Assembly: Independent operation of drill press',
    'Utilizes procedure for bonding tasks',
    'Mechanical Assembly : Utilizes proper procedure for bonding tasks',
    'Assembles wiggle plates',
    'Mechanical Assembly : Assembly of wiggle plates',
    'Sets up minor jobs (nut plate, wiggle plate, countersink machine)',
    'Mechanical Assembly : Independent set-up of minor jobs (nut plate, wiggle plate, countersink machine)',
    'Determines drawings required',
    'QA: Ability to determine drawings required and current revisions',
    'Operates the blanking press',
    'Blanking Press: Acceptable operation of blanking press',
    'Locates tooling for each job|Sets up and takes down tooling|Puts tool away',
    'Blanking Press: Ability to locate proper tooling for each job',
    'Identifies burrs|Deburrs parts',
    'Blanking Press: Correctly deburrs parts',
    'Lubricates tools',
    'Blanking Press: Proper lubrication of material',
    'Operates the Amada',
    'Amada: Proper operation of Amada',
    'Operates brake form in Mylar cutting',
    'Brake Form: Operation of brake form in Mylar cutting',
    'Sets up and centers Mylar cutting tool',
    'Brake Form: Independent ability to set up and center Mylar cutting tool',
    'Identifies burrs|Uses "Timesaver"|Deburrs parts',
    'Brake Form: Correctly deburrs parts',
    'Identifies and locates correct material for each job|Loads and unloads material|Performs computer steps to start machine',
    'Routers: Independent ability to identify and locate correct material for each job',
    'Identifies and locates tooling for each job|Sets up and changes tooling plates',
    'Routers: Independent ability to setup and change tooling plates',
    'Identifies burrs|Deburrs parts',
    'Routers: Correctly deburrs parts',
    'Differentiates between Boeing and Commercial work orders|Locates tools and templates used for Boeing work orders|Locates, prints, and verifies certification (cert)',
    'Shears: Independent ability to differentiate between Boeing and Commercial work orders',
    'Sets up the machine',
    'Shears: Independent ability to set up machine',
    'Operates the shear',
    'Shears: Accurate operation of Shear',
    'Understands machine operation and computer inputs|Differentiates between Automatic Start and Manual Start procedure and when each would be used',
    'Waterjet: Understands and demonstrates proper machine operation and computer inputs',
    'Operates the water jet|Sets up the water jet',
    'Waterjet: Independent setup and machine operation of Water Jet',
    'Operates the grind wheel|Uses deburr hand tools including scotch brite or sand paper|Cleans parts thoroughly with alcohol',
    'Deburr: Cleans parts thoroughly with alcohol',
    'Counts and verifies part mark and paperwork prior to packaging outgoing shipment',
    'Commercial Shipping: Sort, segregate and prioritize verified work order by due date and shipping date',
    'Visually inspects incoming product for damage, re-package product according to Orion guidelines prior to moving on to next area',
    'Receiving: Independence with receiving materials and performing tasks in Receiving work area',
    'Operates the pallet jack',
    'Receiving: Independent utilization of pallet jack',
    'Visually inspects incoming product for damage, re-packages product per Orion guidelines prior to moving into inventory',
    "Inventory: for parts pulled out of inventory, verifies correct part #'s on work order",
    'Retrieves product from inventory; moves to next location based on work order or pull ticket',
    'Inventory: Independently finds correct location when picking parts from inventory',
    'Counts product in each specified bin location during cycle count activity',
    'Inventory: Ensures that counts in inventory match what is posted for cycle counts',
    'Applies bond and adheres waffle or strips to orifice|Places weights on waffle orifices',
    'Bond: Application of bond and adherence of waffle or strips to orifice',
    'Calculates cure time for bond',
    'Bond: Independent, accurate calculation of cure time for bond',
    'Creates internal work order',
    'Shipping & Receiving: Independent creation of internal work order',
    'Enters information into Boeing Portal',
    'Shipping & Receiving: Accurate entering of information into Boeing Portal',
    'Utilizes and navigates computer applications',
    'Manufacturing Offices: Independent ability to utilize and navigate computer applications',
    'Creates Purchase Orders|Creates quotes/estimates',
    'Manufacturing Offices: Independently creates Purchase Orders',
    'Gathers data from internal lists and makes determinations based on information',
    'Manufacturing Offices: Gathers correct data from internal lists and makes correct decisions based on this informaton',
    'Visually inspects and measures parts',
    'Wire Assembly: Independent visual inspection and measurement of each part',
    'Ensures that counts per bag match what is posted on front of boxes|Properly runs 2 bag system',
    'Inventory: Proper run of 2 bag system',
    'Inspects and verifies wire part marks/lengths|Completes job (operations closed, paperwork stamped, etc.)',
    'Wire QA: Accurate inspection and verification of wire lengths',
    'Loads and threads wire through de-reeler, unracks wire spool and returns to its correct location',
    'Laser Marking: Independently and accurately loads and threads wire through de-reeler',
    'Follows procedure to access laser marking information on flash drive/computer',
    'Laser Marking: Follows proper procedure to access laser marking information on flash drive/computer',
    'location code']].copy()

In [258]:
evaldf4.to_excel(r'redacted file path\exports\eval merge 6.xlsx')

### evaluation 2

In [71]:
val = eval_df3.copy()

In [72]:
soft = val[val['Mentor Evaluation Status'].notna().copy()]
hard = val[val['Mentor Evaluation Status'].isna().copy()]
soft.info(),hard.info()

KeyError: 'Mentor Evaluation Status'

In [177]:
# soft.to_excel(r'redacted file path\exports\evalsoft.xlsx')
# hard.to_excel(r'redacted file path\exports\evalhard.xlsx')

In [22]:
soft = soft.drop(soft.columns[9:959],axis=1)
hard = hard.drop(hard.columns[959:986],axis=1)
soft.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10827 entries, 12360 to 23186
Data columns (total 40 columns):
 #   Column                                                                                                                         Non-Null Count  Dtype 
---  ------                                                                                                                         --------------  ----- 
 0   Venue                                                                                                                          10824 non-null  object
 1   Mentor                                                                                                                         10827 non-null  object
 2   Individual                                                                                                                     10827 non-null  object
 3   Work Cell                                                                                                       

### Demographics Merge

In [116]:
# peeps3 is the base merge form that contains first last and birthday to prevent duplicates

# peeps3

In [109]:
demo1 = pd.read_excel(r'redacted file path\Client information (demographics and assessment) - unformatted.xlsx')

# sort the individual column values alphabetically
demo1 = demo1.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
demo1['Individual'] = demo1['Individual'].str.replace(', ',',')

# remove test case rows
demo1 = demo1[~demo1['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
demo1['Individual'] = demo1['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

demo1['Individual'] = demo1['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

demomerge = peeps3.merge(demo1,how='left',on='Individual')
demomerge = demomerge.dropna(how='all',axis=1)
demomerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1391 entries, 0 to 1390
Data columns (total 57 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   field_3436_first                          1390 non-null   object        
 1   field_3436_last                           1390 non-null   object        
 2   field_3195                                1390 non-null   object        
 3   Individual                                1391 non-null   object        
 4   ID                                        1305 non-null   float64       
 5   Preferred Names                           635 non-null    object        
 6   Date of Birth                             1210 non-null   datetime64[ns]
 7   Gender                                    1250 non-null   object        
 8   Disability - Primary                      1028 non-null   object        
 9   Disabilities - Additional     

In [110]:
demomerge2 = demomerge.drop_duplicates(subset=['field_3436_first','field_3436_last','field_3195'])

In [111]:
import re


demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].replace(r';#\d{2}','|',regex=True)
demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].str.rstrip('|')
demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].str.replace(';#','')
demomerge2['Ethnicity'].head(50)

0                              African American / Black
1                                                 Other
2                                     Caucasian / White
3                                     Caucasian / White
4                                     Caucasian / White
5                                     Caucasian / White
6                                                   NaN
7                                     Caucasian / White
8                                     Caucasian / White
9                                     Caucasian / White
10                                    Caucasian / White
11                                    Caucasian / White
12                             African American / Black
13                                             Hispanic
14    African American / Black|Native Hawaiin / Othe...
15                                    Caucasian / White
16                                    Caucasian / White
17              Native Hawaiin / Other Pacific I

In [112]:
demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].str.replace('Hawaiin','Hawaiian')
demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].str.replace('African American / Black','Black / African American')
demomerge2.loc[:,'Ethnicity'] = demomerge2['Ethnicity'].str.replace('Multi-Racial','Other')

In [113]:
demomerge2['Ethnicity'].value_counts()

Ethnicity
Caucasian / White                                                                                    727
Black / African American                                                                             155
Asian                                                                                                 93
Other                                                                                                 36
Hispanic                                                                                              35
Native Hawaiian / Other Pacific Islander                                                              28
Unknown                                                                                               19
American Indian / Alaskan Native                                                                      15
Caucasian / White|American Indian / Alaskan Native                                                    10
Black / African American|Caucasian / White   

In [198]:
demomerge2.to_excel(r'redacted file path\exports\demographics import2.xlsx')

### Intake Assessment

In [114]:
demo1 = pd.read_excel(r'redacted file path\Client information (demographics and assessment) - unformatted.xlsx')

In [115]:
# sort the individual column values alphabetically
demo1 = demo1.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
demo1['Individual'] = demo1['Individual'].str.replace(', ',',')

# remove test case rows
demo1 = demo1[~demo1['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
demo1['Individual'] = demo1['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

demo1['Individual'] = demo1['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [116]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [117]:
assess = demo1.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')
assess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 92 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   ID                                        520 non-null    float64       
 1   Individual                                520 non-null    object        
 2   Preferred Names                           144 non-null    object        
 3   Date of Birth                             486 non-null    datetime64[ns]
 4   Gender                                    512 non-null    object        
 5   Disability - Primary                      414 non-null    object        
 6   Disabilities - Additional                 317 non-null    object        
 7   Disability / Barrier Notes:               357 non-null    object        
 8   Date of Last Employment                   362 non-null    datetime64[ns]
 9   Smoker?                         

In [118]:
assess['disabilities'] = assess['Disability - Primary'] +' '+ assess['Disabilities - Additional'] + ' '+ assess['Disability / Barrier Notes:']
assess['disabilities'] = assess['disabilities'].astype(str)

In [119]:
assess['allergy'] = np.where(assess['Allergies'].notna(),'Yes','No')

In [120]:
mentallist = ['Depression','Anxiety','PTSD','ADHD','Substance Abuse','Learning Disability','Hearing','asthma','vision','memory','seizure']

for i in mentallist:
    assess[i] = assess['disabilities'].apply(lambda x: 'Yes' if i in x else 'No')

In [121]:
assess['Transportation'].value_counts()

Transportation
Car               206
Public Transit    170
Para-transit       38
Ride               28
Other              15
School Bus         10
Unknown             4
Bike                3
Name: count, dtype: int64

In [122]:
tranrep = {
    'Car':'Driving Myself',
    'Public Transit':'Public Transportation',
    'Para-transit':'Paratransit',
    'Ride':'Rides/Carpool',
    'Unknown':'Other'
}
assess['Transportation'] = assess['Transportation'].replace(tranrep)

In [123]:
assess['Work Restrictions'] =assess['Work Restrictions'].astype(str)

physlist = ['Standing','Sitting','Lift over 10','Lift over 25','Bending / Stooping']

for i in physlist:
    assess[i] = assess['Work Restrictions'].apply(lambda x: 'No' if i in x else 'Yes')

In [124]:
mentlist = ['Reading','English Language Proficiency',]

for i in mentlist:
    assess[i] = assess['Work Restrictions'].apply(lambda x: 'Yes' if i in x else 'No')

In [125]:
assess['Education Level'].value_counts()

Education Level
High School Diploma                                           215
Some College / Technical School (no certificate / diploma)     67
No High School Diploma / GED                                   58
GED                                                            57
Graduated - In High School                                     29
Bachelors                                                      25
Associates                                                     21
In High School                                                 12
Certificate                                                     6
Other                                                           4
Masters (or beyond)                                             3
Name: count, dtype: int64

In [126]:
edurep = {
    'High School Diploma':'HS Diploma',
    'Some College / Technical School (no certificate / diploma)':'Some post high school',
    'No High School Diploma / GED':'No HS Diploma - Under 12th grade education',
    'Graduated - In High School':'HS Diploma',
    'Masters (or beyond)':'Masters and above',
    'In High School':'No HS Diploma - Under 12th grade education',
    'Certificate':'Certificate - 2 years or less'
}
assess['Education Level'] = assess['Education Level'].replace(edurep)

In [127]:
assess['Benefits'] = assess['Benefits'].str.replace(';#',',',regex=True)
assess['Benefits'] = assess['Benefits'].str.replace('Food Stamps','SNAP',regex=True)
assess['Benefits'] = assess['Benefits'].str.replace('None','',regex=True)

In [128]:
assess['Insurance'] = np.where(assess['Benefits'].str.contains('Medical Benefits \(State / Fed\)'),'State Health Insurance for Adults','')

assess['non cash'] = np.where(assess['Benefits'].str.contains('Childcare Voucher'),'Childcare Voucher',
                            np.where(assess['Benefits'].str.contains('Housing Choice'),'Housing Choice Voucher',
                            np.where(assess['Benefits'].str.contains('Public Housing'),'Public Housing',
                            np.where(assess['Benefits'].str.contains('Permanent Supportive'),'Permanent Supportive Housing',
                            ''))))

In [129]:
benrep = {
    'L&I - Time Loss':'L&I Time Loss',
    'SNAP':'SNAP / EBT / Food Stamps',
    'Unemployment Insurance':'Unemployment',
    'Veterans Benefits':'VA Service-Connectid Disability Compensation',
    'WIC (CSBG)':'WIC',
    'Medical Benefits \(State / Fed\)':'',
    'Housing Assistance':'',
    'Childcare Voucher \(CSBG\)':'',
    'Housing Choice Voucher \(CSBG\)':'',
    'Public Housing \(CSBG\)':'',
    'Permanent Supportive Housing \(CSBG\)':''
}

assess['Benefits'] = assess['Benefits'].replace(benrep,regex=True)

In [130]:
assess['medication'] = np.where(assess['Medications and Conditions'].notna(),'Yes','No')

In [131]:
assess['Benefits']

0                                ,,SSI
1                                 SSDI
2                                     
3            ,SNAP / EBT / Food Stamps
4             SNAP / EBT / Food Stamps
                    ...               
530      SNAP / EBT / Food Stamps,,SSI
531           SNAP / EBT / Food Stamps
532       SNAP / EBT / Food Stamps,SSI
533    ,,SNAP / EBT / Food Stamps,SSDI
534          ,SNAP / EBT / Food Stamps
Name: Benefits, Length: 535, dtype: object

In [132]:
assess['Benefits'] = assess['Benefits'].str.replace(',,',',',regex=True)
assess['Benefits'] = assess['Benefits'].str.strip(',')

In [133]:
assess['Benefits'] = assess['Benefits'].str.replace(',','|',regex=True)
assess['Benefits']

0                                SSI
1                               SSDI
2                                   
3           SNAP / EBT / Food Stamps
4           SNAP / EBT / Food Stamps
                   ...              
530     SNAP / EBT / Food Stamps|SSI
531         SNAP / EBT / Food Stamps
532     SNAP / EBT / Food Stamps|SSI
533    SNAP / EBT / Food Stamps|SSDI
534         SNAP / EBT / Food Stamps
Name: Benefits, Length: 535, dtype: object

In [134]:
assess['Child'] = np.where((assess['Number of Dependents'] == 0) | (assess['Number of Dependents'].isna()),'No','Yes')

In [135]:
assess2 = assess[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Created',
    'Standing',
    'Sitting',
    'Bending / Stooping',
    'Lift over 10',
    'Lift over 25',
    'disabilities',
    'Education Level',
    'Reading',
    'English Language Proficiency',
    'Special Education History',
    'Hearing',
    'vision',
    'memory',
    'seizure',
    'asthma',
    'allergy',
    'Allergies',
    'Depression',
    'Anxiety',
    'PTSD',
    'disabilities',
    'ADHD',
    'medication',
    'Medications and Conditions',
    'Transportation',
    'Child',
    'Household Size',
    'Number of Dependents',
    'Substance Abuse',
    'Smoker?',
    'Benefits',
    'Insurance',
    'Health Insurance',
    'non cash'
]]

In [163]:
assess2.to_excel(r'redacted file path\exports\real assess merge3.xlsx')

### Case note merge

In [136]:
case1 = pd.read_excel(r'redacted file path\case note - unformatted.xlsx')
case1 = case1.add_suffix('_case')
case2 = pd.read_excel(r'redacted file path\outcome activity (case note) - unformatted.xlsx')
case2 = case2.add_suffix('_activity')
area = pd.read_excel(r'redacted file path\outcome area (case notes) - unformatted.xlsx')
area = area.add_suffix('_area')
goal = pd.read_excel(r'redacted file path\outcome goal (goals) - unformatted.xlsx')
goal = goal.add_suffix('_goal')

In [137]:
outcome1 = goal.merge(area,how='left',left_on='Outcome Area ID_goal',right_on='ID_area')
outcome1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5885 entries, 0 to 5884
Data columns (total 44 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Start Date_goal                               5865 non-null   datetime64[ns]
 1   Goal_goal                                     5882 non-null   object        
 2   Action Steps_goal                             0 non-null      float64       
 3   Goal Summary_goal                             1830 non-null   object        
 4   Created_goal                                  5885 non-null   datetime64[ns]
 5   Created By_goal                               5885 non-null   object        
 6   Modified_goal                                 5885 non-null   datetime64[ns]
 7   Modified By_goal                              5885 non-null   object        
 8   Client_goal                                   5885 non-null   object

In [138]:
outcome2 = case2.merge(outcome1,how='left',left_on='GoalID_activity',right_on='ID_goal')

In [139]:
outcome2.columns

Index(['Date_activity', 'Client_activity',
       'Time Spent on this Activity_activity', 'Client Hours_activity',
       'Hours Type_activity', 'Staff_activity', 'Contact Type_activity',
       'DDA Phase_activity', 'Modified_activity', 'Activity_activity',
       'Administrative Time_activity', 'Progress_activity',
       'TotalHours_activity', 'Travel Time_activity', 'CaseNoteID_activity',
       'Content Type_activity', 'Created_activity', 'Created By_activity',
       'Folder Child Count_activity', 'GoalID_activity', 'ID_activity',
       'Item Child Count_activity', 'Modified By_activity', 'Title_activity',
       'Item Type_activity', 'Path_activity', 'Start Date_goal', 'Goal_goal',
       'Action Steps_goal', 'Goal Summary_goal', 'Created_goal',
       'Created By_goal', 'Modified_goal', 'Modified By_goal', 'Client_goal',
       'Completion Date_goal', 'Discontinuation Date_goal',
       'Content Type_goal', 'Folder Child Count_goal',
       'How do you know this goal has been 

In [140]:
outcome3 = outcome2[[
    'Client_activity',
    'Date_activity',
    'Contact Type_activity',
    'Time Spent on this Activity_activity',
    'Activity_activity',
    'DDA Phase_activity',
    'Administrative Time_activity',
    'Travel Time_activity',
    'Goal_goal',
    'Service_area',
    'Funding Source_area'
]]

In [141]:
case1.columns

Index(['ID_case', 'Date_case', 'Client_case', 'Venue_case', 'Service_case',
       'DDA Phase_case', 'Contact Type_case', 'Created_case',
       'Created By_case', 'Content Type_case',
       'Hours Spent (not on defined goals)_case', 'Other Comments_case',
       'Administrative Time_case', 'Folder Child Count_case',
       'Funding Source_case', 'Item Child Count_case', 'Modified_case',
       'Modified By_case', 'Staff_case', 'Title_case',
       'Total Hours Spent (not on defined goals)_case', 'Travel Time_case',
       'Item Type_case', 'Path_case'],
      dtype='object')

In [142]:
case11 = case1[[
    'Client_case',
    'Date_case',
    'Contact Type_case',
    'Hours Spent (not on defined goals)_case',
    'Other Comments_case',
    'DDA Phase_case',
    'Administrative Time_case',
    'Travel Time_case',
    'Service_case',
    'Funding Source_case'
]]

In [143]:
casecol1 = [
    'Client',
    'Date',
    'Contact Type',
    'Time Spent',
    'Case Note',
    'DDA Phase',
    'Admin Time',
    'Travel Time',
    'Service',
    'Funding'
]

casecol2 = [
    'Client',
    'Date',
    'Contact Type',
    'Time Spent',
    'Case Note',
    'DDA Phase',
    'Admin Time',
    'Travel Time',
    'Goal',
    'Service',
    'Funding'
]

case11.columns = casecol1
outcome3.columns = casecol2

In [144]:
casenote = pd.concat([outcome3,case11],axis=0)
casenote.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81884 entries, 0 to 47771
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Client        81884 non-null  object        
 1   Date          81884 non-null  datetime64[ns]
 2   Contact Type  72346 non-null  object        
 3   Time Spent    66937 non-null  float64       
 4   Case Note     72315 non-null  object        
 5   DDA Phase     11666 non-null  object        
 6   Admin Time    2 non-null      float64       
 7   Travel Time   6 non-null      float64       
 8   Goal          34022 non-null  object        
 9   Service       75150 non-null  object        
 10  Funding       68276 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 7.5+ MB


In [145]:
casenote['Goal Reviewed?'] = np.where(casenote['Goal'].notna(),'Yes','No')

In [146]:
casenote[['Client','Individual']] = casenote['Client'].str.split(';#', expand=True)

# sort the individual column values alphabetically
casenote = casenote.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
casenote['Individual'] = casenote['Individual'].str.replace(', ',',')

# remove test case rows
casenote = casenote[~casenote['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
casenote['Individual'] = casenote['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

casenote['Individual'] = casenote['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [147]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [148]:
casenote2 = casenote.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')
casenote2 = casenote2.drop(columns=['Admin Time','Travel Time'])
casenote2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50996 entries, 0 to 50995
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Client              50990 non-null  object        
 1   Date                50990 non-null  datetime64[ns]
 2   Contact Type        43753 non-null  object        
 3   Time Spent          41640 non-null  float64       
 4   Case Note           43783 non-null  object        
 5   DDA Phase           10306 non-null  object        
 6   Goal                19298 non-null  object        
 7   Service             48021 non-null  object        
 8   Funding             46442 non-null  object        
 9   Goal Reviewed?      50990 non-null  object        
 10  Individual          50990 non-null  object        
 11  people_First Name   50996 non-null  object        
 12  people_Last Name    50996 non-null  object        
 13  demo_Date of Birth  50996 non-null  object    

In [149]:
casenote2['Time Spent'].value_counts()

Time Spent
0.25    19339
0.50     7105
1.00     4178
1.50     2566
2.00     2357
0.75     1958
1.25     1394
1.75      849
2.50      664
3.00      391
2.25      260
2.75      136
0.00       91
3.50       67
4.50       52
3.25       45
4.00       40
4.25       34
4.75       26
5.00       23
3.75       16
5.50       12
7.00        6
6.50        6
8.00        5
5.25        5
6.00        4
6.25        3
5.75        3
7.50        3
7.25        2
Name: count, dtype: int64

In [150]:
contactrep = {
    'Face to Face':'In Person',
    'General Info':'General',
    'Instructor / Set-Up':'Instructor',
    'DDA - Record Keeping':'DDA Record Keeping',
    'Referral Agency':'Referral',
    'Time Off Request':'Mentor Communication',
    'Closure Note (Include any return requirements or recommendations)':'General',
    '(Select)':'General'
}
casenote2['Contact Type'] = casenote2['Contact Type'].replace(contactrep)

In [151]:
casenote2['Who Participated'] = 'Client'

In [152]:
casenote2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50996 entries, 0 to 50995
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Client              50990 non-null  object        
 1   Date                50990 non-null  datetime64[ns]
 2   Contact Type        43753 non-null  object        
 3   Time Spent          41640 non-null  float64       
 4   Case Note           43783 non-null  object        
 5   DDA Phase           10306 non-null  object        
 6   Goal                19298 non-null  object        
 7   Service             48021 non-null  object        
 8   Funding             46442 non-null  object        
 9   Goal Reviewed?      50990 non-null  object        
 10  Individual          50990 non-null  object        
 11  people_First Name   50996 non-null  object        
 12  people_Last Name    50996 non-null  object        
 13  demo_Date of Birth  50996 non-null  object    

In [153]:
casenote2['Referrals Made'] = np.where(casenote2['Contact Type'] == 'Referral','Yes','No')

In [154]:
casenote2['Service'].value_counts()

Service
34;#DDA - Individual Employment Services    9320
0;#                                         8618
1;#Assessment                               8229
7;#Job Development                          5053
13;#Community Based Assessment              4843
39;#Case Management                         2617
35;#BFET - SL (Life Skills)                 2395
24;#DDA - Community Inclusion               2092
8;#Job Retention                            1496
12;#Training                                1246
32;#DDA - School to Work                     842
38;#BFET - JT (Job Search Training)          454
3;#Intensive Training Services               263
36;#BFET - JS (Supervised Job Search)        219
37;#BFET - BR (Job Retention)                147
40;#DDA - Job Foundations                     81
6;#Internship - Other                         52
4;#Internship - CNC Mills                     30
23;#IL Services                               23
28;#                                           1
Name: count,

In [155]:
import re
casenote2.loc[:,'Service'] = casenote2['Service'].replace(r'\d{1,2};#','',regex=True)
casenote2.loc[:,'Funding'] = casenote2['Funding'].replace(r'\d{1,2};#','',regex=True)

In [156]:
casenote2['Import Service'] = casenote2['Service']
casenote2['Import Funding'] = casenote2['Funding']

In [157]:
casenote3 = casenote2[casenote2['Case Note'].notna()].copy()
casenote3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43783 entries, 0 to 50995
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Client              43783 non-null  object        
 1   Date                43783 non-null  datetime64[ns]
 2   Contact Type        43548 non-null  object        
 3   Time Spent          41521 non-null  float64       
 4   Case Note           43783 non-null  object        
 5   DDA Phase           10186 non-null  object        
 6   Goal                19153 non-null  object        
 7   Service             40826 non-null  object        
 8   Funding             39259 non-null  object        
 9   Goal Reviewed?      43783 non-null  object        
 10  Individual          43783 non-null  object        
 11  people_First Name   43783 non-null  object        
 12  people_Last Name    43783 non-null  object        
 13  demo_Date of Birth  43783 non-null  object        


In [158]:
query = pd.read_excel(r'redacted file path\exports\appricot subservice funder export.xlsx')

In [159]:
query = query.drop_duplicates(subset=['First','Last','Status','Location','Date Intake Started','Date Services Starting','Service','Sub-Service Category','Funding Source'])
query.info()

<class 'pandas.core.frame.DataFrame'>
Index: 587 entries, 0 to 640
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   First                   587 non-null    object 
 1   Middle                  0 non-null      float64
 2   Last                    587 non-null    object 
 3   Date of Birth           587 non-null    object 
 4   Status                  587 non-null    object 
 5   Location                587 non-null    object 
 6   Date Intake Started     585 non-null    object 
 7   Date Services Starting  587 non-null    object 
 8   Service                 587 non-null    object 
 9   Sub-Service Category    587 non-null    object 
 10  Funding Source          587 non-null    object 
 11  Start Date              587 non-null    object 
 12  associated service      587 non-null    object 
 13  sub service             587 non-null    object 
dtypes: float64(1), object(13)
memory usage: 68.8+ K

In [160]:
casenote3['Apricot Service'] = np.where(casenote3['Import Service'].str.contains('Assessment|Trial Work Experience|Life Skills'),'Assessment',
                                    np.where(casenote3['Import Service'].str.contains('DDA'),'DDA Services',
                                    np.where(casenote3['Import Service'].str.contains('Job Retention|Intensive Training Services'),'Job Retention',
                                    np.where(casenote3['Import Service'].str.contains('Job Development|Supervised Job Search|Job Search Training'),'Job Development',
                                    np.where(casenote3['Import Service'].str.contains('Case Management'),'Case Management',
                                    np.where(casenote3['Import Service'].str.contains('IL Services'),'Independent Living',
                                    np.where(casenote3['Import Service'].str.contains('Internship|Training'),'Training',
                                    '')))))))

In [162]:
casenote3['Import Funding'] = casenote3['Import Funding'].str.replace('1','',regex=True)

In [163]:
fundrep = {
    'Olsen & Associates':'Labor & Industries',
    'Strategic Consulting Services':'Labor & Industries',
    'Pioneer Industries':'Labor & Industries',
    'Aerospace Joint Apprenticeship Committee (AJAC)':'Labor & Industries',
    'Workforce Training and Education Coordinating Board':'WRT',
    'WRT - State Board for Community and Technical Colleges':'WRT',
    'City of Renton':'City Funding - Human Services (Auburn, Renton)',
    'City of Kent':'City Funding - Human Services (Auburn, Renton)',
    'City of Auburn':'City Funding - Human Services (Auburn, Renton)',
    'City of Federal Way':'City Funding - CDBG (Federal Way)',
    'Snohomish County Human Services | Housing and Community Services':'Snohomish County - ERSS',
    'DDA-King County':'DDA',
    'DDA-Snohomish':'DDA',
    'Veterans Administration-Seattle':'Veterans Administration',
    'King County Jobs and Housing':'King County Jobs & Housing',
    'SJI (Seattle Jobs Initiative)':'Labor & Industries',
    'DSB-Tacoma':'DVR'
}

casenote3['Apricot Funding'] = casenote3['Import Funding'].replace(fundrep)

In [164]:
casenote3['Apricot Funding'] = casenote3['Apricot Funding'].str.replace(r'^DVR*','DVR',regex=True)

In [165]:
casenote3['Apricot Funding'].value_counts()

Apricot Funding
DDA                                               11963
                                                   6546
BFET                                               3569
King County Jobs & Housing                         3051
Snohomish County - ERSS                            2726
DVR-Lynnwood                                       1819
DVR-Seatac                                         1517
DVR-Kent                                           1457
City Funding - Human Services (Auburn, Renton)     1417
DVR-Tacoma                                         1081
City Funding - CDBG (Federal Way)                   981
DVR-Everett                                         914
DVR-Puyallup                                        814
WRT                                                 535
DVR-Bellevue                                        413
DVR-Arlington                                       174
DVR-North Seattle                                   156
Labor & Industries              

In [166]:
casenote4 = casenote3.merge(query,how='left',left_on=['people_First Name','people_Last Name','Apricot Service','Apricot Funding'],right_on=['First','Last','Service','Funding Source'])

In [167]:
casenote4['CI Phase'] = np.where(casenote4['Import Service'].str.contains('Community Inclusion'),'Record Keeping','')

In [92]:
casenote4.to_excel(r'redacted file path\exports\real casenote2.xlsx')

### Outcome Area/goal/activity

In [168]:
area = pd.read_excel(r'redacted file path\outcome area (case notes) - unformatted.xlsx')
area = area.add_suffix('_area')
goal = pd.read_excel(r'redacted file path\outcome goal (goals) - unformatted.xlsx')
goal = goal.add_suffix('_goal')
activity = pd.read_excel(r'redacted file path\outcome activity (case note) - unformatted.xlsx')
activity = activity.add_suffix('_activity')

In [169]:
outcome1 = goal.merge(area,how='left',left_on='Outcome Area ID_goal',right_on='ID_area')

outcome1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5885 entries, 0 to 5884
Data columns (total 44 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Start Date_goal                               5865 non-null   datetime64[ns]
 1   Goal_goal                                     5882 non-null   object        
 2   Action Steps_goal                             0 non-null      float64       
 3   Goal Summary_goal                             1830 non-null   object        
 4   Created_goal                                  5885 non-null   datetime64[ns]
 5   Created By_goal                               5885 non-null   object        
 6   Modified_goal                                 5885 non-null   datetime64[ns]
 7   Modified By_goal                              5885 non-null   object        
 8   Client_goal                                   5885 non-null   object

In [170]:
outcome2 = activity.merge(outcome1,how='left',left_on='GoalID_activity',right_on='ID_goal')

In [171]:
outcome2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34112 entries, 0 to 34111
Data columns (total 70 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Date_activity                                 34112 non-null  datetime64[ns]
 1   Client_activity                               34112 non-null  object        
 2   Time Spent on this Activity_activity          32017 non-null  float64       
 3   Client Hours_activity                         32 non-null     float64       
 4   Hours Type_activity                           37 non-null     object        
 5   Staff_activity                                34109 non-null  object        
 6   Contact Type_activity                         33742 non-null  object        
 7   DDA Phase_activity                            6112 non-null   object        
 8   Modified_activity                             34112 non-null  date

In [172]:
# split out the contents of the client column on the ;# and add the split values to the newly created individual column
outcome2[['Client_activity','Individual']] = outcome2['Client_activity'].str.split(';#', expand=True)

# sort the individual column values alphabetically
outcome2 = outcome2.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
outcome2['Individual'] = outcome2['Individual'].str.replace(', ',',')

# remove test case rows
outcome2 = outcome2[~outcome2['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
outcome2['Individual'] = outcome2['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

outcome2['Individual'] = outcome2['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [173]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [174]:
outcome3 = outcome2.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')

In [541]:
for i in outcome3.columns:
    print(i)

Date_activity
Client_activity
Time Spent on this Activity_activity
Client Hours_activity
Hours Type_activity
Staff_activity
Contact Type_activity
DDA Phase_activity
Modified_activity
Activity_activity
Administrative Time_activity
Progress_activity
TotalHours_activity
Travel Time_activity
CaseNoteID_activity
Content Type_activity
Created_activity
Created By_activity
Folder Child Count_activity
GoalID_activity
ID_activity
Item Child Count_activity
Modified By_activity
Title_activity
Item Type_activity
Path_activity
Start Date_goal
Goal_goal
Action Steps_goal
Goal Summary_goal
Created_goal
Created By_goal
Modified_goal
Modified By_goal
Client_goal
Completion Date_goal
Discontinuation Date_goal
Content Type_goal
Folder Child Count_goal
How do you know this goal has been met?_goal
ID_goal
Item Child Count_goal
Outcome Area ID_goal
Projected Completion Date_goal
Reason for Discontinuation_goal
Review Schedule_goal
Title_goal
Item Type_goal
Path_goal
Date_area
Client_area
Service_area
Objecti

In [36]:
outcome3.to_excel(r'redacted file path\exports\outcome area goal activity.xlsx')

#### Goals alone

In [175]:
goalserv = pd.read_excel(r'redacted file path\exports\appricot services export.xlsx')

In [176]:
# split out the contents of the client column on the ;# and add the split values to the newly created individual column
goal[['Client_goal','Individual']] = goal['Client_goal'].str.split(';#', expand=True)

# sort the individual column values alphabetically
goal = goal.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
goal['Individual'] = goal['Individual'].str.replace(', ',',')

# remove test case rows
goal = goal[~goal['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
goal['Individual'] = goal['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

goal['Individual'] = goal['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [177]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [178]:
goalserv['full'] = goalserv['Last']+','+goalserv['First']

In [179]:
goal2 = goal.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')
goal3 = goal2.merge(goalserv,how='right',left_on='Individual',right_on='full')
goal3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2807 entries, 0 to 2806
Data columns (total 36 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Start Date_goal                               2744 non-null   datetime64[ns]
 1   Goal_goal                                     2750 non-null   object        
 2   Action Steps_goal                             0 non-null      float64       
 3   Goal Summary_goal                             507 non-null    object        
 4   Created_goal                                  2752 non-null   datetime64[ns]
 5   Created By_goal                               2752 non-null   object        
 6   Modified_goal                                 2752 non-null   datetime64[ns]
 7   Modified By_goal                              2752 non-null   object        
 8   Client_goal                                   2752 non-null   object

In [180]:
goal3['associated service'] = goal3['Status']+' - '+goal3['Location']+' - Enrolled: '+goal3['Date Services Starting']
goal3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2807 entries, 0 to 2806
Data columns (total 37 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Start Date_goal                               2744 non-null   datetime64[ns]
 1   Goal_goal                                     2750 non-null   object        
 2   Action Steps_goal                             0 non-null      float64       
 3   Goal Summary_goal                             507 non-null    object        
 4   Created_goal                                  2752 non-null   datetime64[ns]
 5   Created By_goal                               2752 non-null   object        
 6   Modified_goal                                 2752 non-null   datetime64[ns]
 7   Modified By_goal                              2752 non-null   object        
 8   Client_goal                                   2752 non-null   object

In [181]:
goal3['Actual goal completion'] = goal3['Completion Date_goal'].combine_first(goal3['Discontinuation Date_goal'])

In [182]:
goal3['goal end reason'] = np.where(goal3['Completion Date_goal'].notna(),'Goal Completed',
                            np.where(goal3['Discontinuation Date_goal'].notna(),'Goal closed - Incomplete',
                                    None))
goal3['goal category'] = None
goal3['goal category'] = goal3['goal category'].fillna('Other Goal Not Listed')
goal3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2807 entries, 0 to 2806
Data columns (total 40 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Start Date_goal                               2744 non-null   datetime64[ns]
 1   Goal_goal                                     2750 non-null   object        
 2   Action Steps_goal                             0 non-null      float64       
 3   Goal Summary_goal                             507 non-null    object        
 4   Created_goal                                  2752 non-null   datetime64[ns]
 5   Created By_goal                               2752 non-null   object        
 6   Modified_goal                                 2752 non-null   datetime64[ns]
 7   Modified By_goal                              2752 non-null   object        
 8   Client_goal                                   2752 non-null   object

In [183]:
goal4 = goal3[[
    'First',
    'Middle',
    'Last',
    'Date of Birth',
    'associated service',
    'Start Date_goal',
    'Projected Completion Date_goal',
    'Actual goal completion',
    'goal end reason',
    'goal category',
    'Goal_goal',
    'Location'
]].copy()

goal4['Location'] = goal4['Location'].replace('Auburn',1174)
goal4['Location'] = goal4['Location'].replace('Mukilteo',2870)

In [184]:
goal4['goal category'] = np.where(goal4['Goal_goal'].str.contains('assess|exhibit|work readiness|increase ability|report income|housing|demonstrate|evaluate|explore|mental health|sobriety|explore a job path|client interests|ptsd|adhd|anxiety|bi-polar|depression|epilepsy|schizo|IOP|legal|relapse|car|driv|license|math',case=False),'Assessment',
                            np.where(goal4['Goal_goal'].str.contains('JD|interview|resume|obtain a job|find a job|obtain employment|obtain competitive employment|full time|full-time|part time|part-time|job search|elevator|develop|intern',case=False),'Job Development',
                            np.where(goal4['Goal_goal'].str.contains('maintain employment|maintain competitive employment|90 day|retain|retention|90+',case=False),'Job Retention',
                            np.where(goal4['Goal_goal'].str.contains('CIS| CI|CI |community inclusion',case=False),'Community Inclusion',
                            np.where(goal4['Goal_goal'].str.contains('EC | EC|independent',case=False),'Individual Employment',
                            np.where(goal4['Goal_goal'].str.contains('CM',case=False),'Case Management',
                            'Other Goal Not Listed'))))))

In [185]:
goal4['goal category'].value_counts()

goal category
Assessment               1941
Job Development           464
Other Goal Not Listed     179
Job Retention             121
Individual Employment      74
Case Management            16
Community Inclusion        12
Name: count, dtype: int64

In [186]:
goal5 = goal4[goal4['Goal_goal'].notna()].copy()
goal5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2750 entries, 0 to 2806
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   First                           2750 non-null   object        
 1   Middle                          0 non-null      float64       
 2   Last                            2750 non-null   object        
 3   Date of Birth                   2750 non-null   object        
 4   associated service              2750 non-null   object        
 5   Start Date_goal                 2744 non-null   datetime64[ns]
 6   Projected Completion Date_goal  1999 non-null   datetime64[ns]
 7   Actual goal completion          1300 non-null   datetime64[ns]
 8   goal end reason                 1300 non-null   object        
 9   goal category                   2750 non-null   object        
 10  Goal_goal                       2750 non-null   object        
 11  Location 

In [677]:
goal5.to_excel(r'redacted file path\exports\real goals2.xlsx')

#### Service Plan

In [187]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [188]:
goalserv = pd.read_excel(r'redacted file path\exports\appricot services export.xlsx')

In [189]:
serve = pd.read_excel(r'redacted file path\exports\real enrollment to service import2.xlsx')

In [190]:
subserve = pd.read_excel(r'redacted file path\exports\appricot subservice funder export.xlsx')

In [191]:
service = pd.read_excel(r'redacted file path\Services - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
service[['Individual','Individual2']] = service['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
service = service.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
service['Individual2'] = service['Individual2'].str.replace(', ',',')

# remove test case rows
service = service[~service['Individual2'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
service['Individual2'] = service['Individual2'].replace(r'\(.*?\)','',regex=True).str.strip()

service['Individual2'] = service['Individual2'].replace(r'\".*?\"','',regex=True).str.strip()

# service['Individual'] = service['Individual2']
# service[['first','last']] = None
service[['last','first','other']] = service['Individual2'].str.split(',', expand=True)

In [193]:
servicemergex = service.merge(subserve,how='left',left_on=['first','last'],right_on=['First','Last'])
servicemergex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3507 entries, 0 to 3506
Data columns (total 38 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Start Date_x                          3507 non-null   datetime64[ns]
 1   Individual                            3507 non-null   object        
 2   Venue                                 3507 non-null   object        
 3   End Date                              3096 non-null   object        
 4   Service_x                             3451 non-null   object        
 5   Recommendations                       439 non-null    object        
 6   Hours Allocated Per Month (DDA Only)  207 non-null    float64       
 7   Modified By                           3507 non-null   object        
 8   Content Type                          3507 non-null   object        
 9   Created                               3507 non-null   datetime64[ns]
 10  

In [194]:
servicemergex.to_excel(r'redacted file path\exports\service plan miss test.xlsx')

In [196]:
servicemerge = service.merge(subserve,how='right',left_on=['first','last'],right_on=['First','Last'])
servicemerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828 entries, 0 to 1827
Data columns (total 38 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Start Date_x                          1820 non-null   datetime64[ns]
 1   Individual                            1820 non-null   object        
 2   Venue                                 1820 non-null   object        
 3   End Date                              1506 non-null   object        
 4   Service_x                             1797 non-null   object        
 5   Recommendations                       212 non-null    object        
 6   Hours Allocated Per Month (DDA Only)  166 non-null    float64       
 7   Modified By                           1820 non-null   object        
 8   Content Type                          1820 non-null   object        
 9   Created                               1820 non-null   datetime64[ns]
 10  

In [197]:
service2 = service.merge(peepsmerge,how='right',left_on='Individual2',right_on='people_Full Name')

In [198]:
service2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1069 entries, 0 to 1068
Data columns (total 28 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Start Date                            1064 non-null   datetime64[ns]
 1   Individual                            1064 non-null   object        
 2   Venue                                 1064 non-null   object        
 3   End Date                              844 non-null    object        
 4   Service                               1046 non-null   object        
 5   Recommendations                       133 non-null    object        
 6   Hours Allocated Per Month (DDA Only)  89 non-null     float64       
 7   Modified By                           1064 non-null   object        
 8   Content Type                          1064 non-null   object        
 9   Created                               1064 non-null   datetime64[ns]
 10  

In [199]:
servicemerge = service2.merge(subserve,how='right',left_on=['first','last'],right_on=['First','Last'])
servicemerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 42 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Start Date_x                          1812 non-null   datetime64[ns]
 1   Individual                            1812 non-null   object        
 2   Venue                                 1812 non-null   object        
 3   End Date                              1509 non-null   object        
 4   Service_x                             1789 non-null   object        
 5   Recommendations                       213 non-null    object        
 6   Hours Allocated Per Month (DDA Only)  151 non-null    float64       
 7   Modified By                           1812 non-null   object        
 8   Content Type                          1812 non-null   object        
 9   Created                               1812 non-null   datetime64[ns]
 10  

In [200]:
# servicemerge.to_excel(r'redacted file path\exports\service plan export.xlsx')

In [201]:
servicemerge2 = servicemerge[[
    'First',
    'Middle',
    'Last',
    'Date of Birth',
    'associated service',
    'sub service',
    'Service_y',
    'Start Date_y',
    'Projected End Date',
    'End Date',
    'Location'
]].copy()

servicemerge2['enrollmentID'] = range(1,len(servicemerge2)+1)

In [202]:
servicemerge2['Service_y'].value_counts()

Service_y
Assessment            671
Job Development       513
Job Retention         313
DDA Services          179
Case Management        82
Training               64
Independent Living      3
Staffing                1
Name: count, dtype: int64

In [617]:
servicemerge2.to_excel(r'redacted file path\exports\service plan export.xlsx')

### Employment placement record

In [203]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [204]:
placement = pd.read_excel(r'redacted file path\Employer placement record - unformatted.xlsx')

placement[['Individal','Individual2']] = placement['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
placement = placement.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
placement['Individual2'] = placement['Individual2'].str.replace(', ',',')

# remove test case rows
placement = placement[~placement['Individual2'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
placement['Individual2'] = placement['Individual2'].replace(r'\(.*?\)','',regex=True).str.strip()

placement['Individual2'] = placement['Individual2'].replace(r'\".*?\"','',regex=True).str.strip()

# remove unnecessary columns
placement = placement.drop(columns=['Content Type','Created By','Item Type','Path'])

In [205]:
placement2 = placement.merge(peepsmerge,how='right',left_on='Individual2',right_on='people_Full Name')
placement2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658 entries, 0 to 657
Data columns (total 29 columns):
 #   Column                                                       Non-Null Count  Dtype         
---  ------                                                       --------------  -----         
 0   Title                                                        347 non-null    object        
 1   Placement Date                                               347 non-null    datetime64[ns]
 2   Orion Staff                                                  347 non-null    object        
 3   Individual                                                   347 non-null    object        
 4   Placement Type                                               344 non-null    object        
 5   Employer                                                     345 non-null    object        
 6   Employer:Company Name                                        345 non-null    object        
 7   Employer Contact 

In [206]:
placement3 = placement2[placement2['Individual'].notna()].copy()

In [207]:
placement3['Placement Type'].value_counts()

Placement Type
Manufacturing - Assembly                         55
Food Service / Restaurant                        48
Customer Service / Retail                        44
Manufacturing - Production                       41
Custodial / Janitorial / Maintenance             29
Manufacturing - Machine Operation / Machining    25
Warehouse                                        22
Administrative / Clerical                        16
Hospitality                                      11
Driver / Shipping & Recieving                    10
Manufacturing - Finish / Paint                    8
Medical (PCA, CNA, Dietary Aide)                  7
Construction / Trade                              7
Other (try to fit in other category)              7
Manufacturing - Composite                         6
Education                                         4
Manufacturing - Quality                           3
Management                                        1
Name: count, dtype: int64

In [208]:
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Warehouse','Warehouse / Shipping & Receiving')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Customer contact / Retail','Retail / Retail Sales')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Administrative / Clerical','Administrative Support Occupations')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Driver / Shipping & Recieving','Driver')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Other (try to fit in other category)','Other Occupations (try to fit in other category)')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Hospitality','Customer contact (e.g. greeter, call center)')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Medical (PCA, CNA, Dietary Aide','Medical/Health Care Occupations (e.g. lab tech)')
placement3['Placement Type'] = placement3['Placement Type'].str.replace('Education','Education (e.g. paraeducator, daycare)')


In [209]:
placement4 = placement3[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Placement Date',
    'End Date',
    'Employer:Company Name',
    'Competitive Employment: Made 90 days?',
    'Customized Employment:',
    'Customized Employment: Wage',
    'Customized Employment: Wage End Date',
    'Customized Employment: Stability Date',
    'Customized Employment: Check if update of current placement'
]].copy()
placement4['customized details'] = placement['Customized Employment: Wage'].astype(str)+'|'+placement4['Customized Employment: Wage End Date'].astype(str)+'|'+placement4['Customized Employment: Stability Date'].astype(str)

In [627]:
# placementmerge = placement.merge(peeps3,how='left',left_on='Individual2',right_on='Individual')
# placementmerge

In [679]:
placement4.to_excel(r'redacted file path\exports\real employment placement2.xlsx')

In [210]:
employment = placement3[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Placement Date',
    'Placement Type',
    'Job Title',
    'Customized Employment: Wage',
    'Competitive Employment: Wage at Placement',
    'Hours worked per week?',
    'Benefits at closure?',
    'Employer:Company Name'
]].copy()

In [681]:
employment.to_excel(r'redacted file path\exports\real employment details3.xlsx')

### Contacts

In [211]:
contact = pd.read_excel(r'redacted file path\Relationships (contacts) - unformatted.xlsx')

# sort the individual column values alphabetically
contact = contact.sort_values('Individual',ascending=True).reset_index()

# the key value had a space in it, so needed to remove that space
contact['Individual'] = contact['Individual'].str.replace(' ','')

contact = contact.drop(columns=['View','Item Type','Path','index','Content Type','Folder Child Count','Item Child Count','Modified By','Modified'])

# remove test case rows
contact = contact[~contact['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
contact['Individual'] = contact['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

contact['Individual'] = contact['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [212]:
contact[['contact last','contact first']] = contact['Contact'].str.split(', ', expand=True)

In [213]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
    'intake_Orion Location'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')

peepsmerge['intake_Orion Location'] = peepsmerge['intake_Orion Location'].replace('Auburn',1174)
peepsmerge['intake_Orion Location'] = peepsmerge['intake_Orion Location'].replace('Mukilteo',2870)

In [214]:
contactmerge = contact.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')

In [215]:
contactmerge2 = contactmerge[contactmerge['Individual'].notna()].copy()

In [216]:
profile = pd.read_excel(r'redacted file path\profiles - unformatted.xlsx')

In [217]:
conpro = contactmerge.merge(profile,how='left',left_on='Contact',right_on='Person')

conpro2 = conpro[conpro['Individual'].notna()].copy()

In [218]:
conpro2.columns

Index(['Individual', 'Contact', 'Relationship to Individual', 'Created_x',
       'Created By_x', 'ID_x', 'contact last', 'contact first',
       'people_First Name', 'people_Last Name', 'demo_Date of Birth',
       'people_Full Name', 'intake_Orion Location', 'ID_y', 'Person',
       'Content Type', 'Programs', 'Organization', 'State', 'City', 'ZIP Code',
       'Active', 'Address', 'Anonymous', 'Became Donor On', 'Board Position',
       'Case Worker', 'Charitable Interest', 'Checkbox Column 1',
       'Checkbox Column 2', 'Choice Column 1', 'Choice Column 2',
       'Choice Column 3', 'Choice Column 4', 'Choice Column 5', 'Comments',
       'Comments Column 1', 'Comments Column 2', 'Country', 'Created_y',
       'Created By_y', 'Date Column 1', 'Date Column 2', 'Donor Motivation',
       'E-Mail', 'Email Lists', 'Folder Child Count', 'Item Child Count',
       'Job Title', 'Link Column 1', 'Link Column 2', 'Mailing Address',
       'Mailing City', 'Mailing Country', 'Mailing Lists',

In [219]:
conpro3 = conpro2[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'contact first',
    'contact last',
    'Relationship to Individual',
    'E-Mail',
    'Phone',
    'Address',
    'City',
    'State',
    'ZIP Code',
    'Professional Type',
    'intake_Orion Location'
]].copy()

In [220]:
conpro3['Active'] = None
conpro3['Active'] = conpro3['Active'].fillna('Active')

In [221]:
conpro3['Emergency'] = np.where(conpro3['Relationship to Individual'].str.contains('Emergency'),'Yes','No')
conpro3['Guardian'] = np.where(conpro3['Relationship to Individual'].str.contains('Guardian'),'Yes','No')
conpro3['Preferred contact'] = np.where(conpro3['Phone'].notna(),'Primary Phone','Address')

In [222]:
conpro3['Relationship to Individual'].value_counts()

Relationship to Individual
Guardian                                                20
Emergency Contact;#Family Member                        19
Counselor                                               19
Case Manager                                            13
Emergency Contact;#Family Member;#Guardian              12
Advocate                                                 8
Family Member                                            4
Residential Provider                                     4
Emergency Contact                                        3
Counselor;#Advocate                                      3
Family Member;#Guardian                                  2
Emergency Contact;#Guardian                              2
Emergency Contact;#Residential Provider                  1
Emergency Contact;#Family Member;#Former Guardian        1
Advocate;#Residential Provider                           1
Former Counselor                                         1
Emergency Contact;#Advocate;#

In [223]:
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.replace('Emergency Contact','',regex=True)
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.replace(';#','|',regex=True)
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.replace('Guardian','Advocate',regex=True)
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.strip('|')
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.replace(r'\|.*','',regex=True)
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].str.replace('Former Counselor','Advocate',regex=True)
conpro3['Relationship to Individual'] = conpro3['Relationship to Individual'].fillna('Advocate')

In [531]:
conpro3.to_excel(r'redacted file path\exports\real contacts2.xlsx')

#### Emergency Contacts

In [224]:
cont = pd.read_excel(r'redacted file path\Client information (demographics and assessment) - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
# cont[['Individual','Individual2']] = cont['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
cont = cont.sort_values('Individual',ascending=True)

# the key value had a space in it, so needed to remove that space
cont['Individual'] = cont['Individual'].str.replace(', ',',')

# remove test case rows
cont = cont[~cont['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
cont['Individual'] = cont['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

cont['Individual'] = cont['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [225]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [226]:
contmerge = cont.merge(peepsmerge,how='right',left_on='Individual',right_on='people_Full Name')
contmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 92 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   ID                                        520 non-null    float64       
 1   Individual                                520 non-null    object        
 2   Preferred Names                           144 non-null    object        
 3   Date of Birth                             486 non-null    datetime64[ns]
 4   Gender                                    512 non-null    object        
 5   Disability - Primary                      414 non-null    object        
 6   Disabilities - Additional                 317 non-null    object        
 7   Disability / Barrier Notes:               357 non-null    object        
 8   Date of Last Employment                   362 non-null    datetime64[ns]
 9   Smoker?                         

In [227]:
emergency = contmerge[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Emergency Contact Name',
    'Emergency Contact Number',
    'Emergency Contact Relationship'
]]

In [228]:
missing = pd.read_excel(r'redacted file path\exports\apricot missing contacts.xlsx')

In [229]:
missingemergency = missing.merge(emergency,how='left', left_on=['First','Last'],right_on=['people_First Name','people_Last Name'])
missingemergency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   First                           480 non-null    object 
 1   Middle                          17 non-null     object 
 2   Last                            480 non-null    object 
 3   Date of Birth                   480 non-null    object 
 4   Record ID                       0 non-null      float64
 5   people_First Name               432 non-null    object 
 6   people_Last Name                432 non-null    object 
 7   demo_Date of Birth              432 non-null    object 
 8   Emergency Contact Name          301 non-null    object 
 9   Emergency Contact Number        297 non-null    object 
 10  Emergency Contact Relationship  297 non-null    object 
dtypes: float64(1), object(10)
memory usage: 41.4+ KB


In [88]:
missingemergency.to_excel(r'redacted file path\exports\missing emergency contacts.xlsx')

### DDA sheets

In [230]:
hours = pd.read_excel(r'redacted file path\Employer placement hours - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
hours[['Individual','Individual2']] = hours['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
hours = hours.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
hours['Individual2'] = hours['Individual2'].str.replace(', ',',')

# remove test case rows
hours = hours[~hours['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
hours['Individual'] = hours['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

hours['Individual'] = hours['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [231]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [232]:
place = pd.read_excel(r'redacted file path\Employer placement record2 - unformatted.xlsx')

In [233]:
hourplace = hours.merge(place,how='left',left_on='EmployerPlacementID',right_on='ID')
hourplace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 48 columns):
 #   Column                                                       Non-Null Count  Dtype         
---  ------                                                       --------------  -----         
 0   Individual_x                                                 920 non-null    object        
 1   Billing Date                                                 912 non-null    datetime64[ns]
 2   Hours worked this MONTH                                      912 non-null    float64       
 3   EmployerPlacementID                                          919 non-null    float64       
 4   Content Type_x                                               920 non-null    object        
 5   Created_x                                                    920 non-null    datetime64[ns]
 6   Created By_x                                                 920 non-null    object        
 7   Folder Child Coun

In [234]:
hourplace2 = hourplace.merge(peepsmerge,how='left',left_on='Individual2',right_on='people_Full Name')
hourplace2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 52 columns):
 #   Column                                                       Non-Null Count  Dtype         
---  ------                                                       --------------  -----         
 0   Individual_x                                                 920 non-null    object        
 1   Billing Date                                                 912 non-null    datetime64[ns]
 2   Hours worked this MONTH                                      912 non-null    float64       
 3   EmployerPlacementID                                          919 non-null    float64       
 4   Content Type_x                                               920 non-null    object        
 5   Created_x                                                    920 non-null    datetime64[ns]
 6   Created By_x                                                 920 non-null    object        
 7   Folder Child Coun

In [38]:
hourplace2.to_excel(r'redacted file path\exports\dda month hours1.xlsx')

In [235]:
ddahourexport = pd.read_excel(r'redacted file path\exports\external linking\employment for dda hours export report.xlsx')

In [236]:
emprep = {
    'Walmart':'Walmart Kelsey St',
    'Safeway':'Safeway Bothell Everett',
    'Home Depot':'Home Depot Hwy 99' 
}

In [237]:
hourplace3 = hourplace2.merge(ddahourexport,how='left',left_on=['people_First Name','people_Last Name','Employer:Company Name'],right_on=['First','Last','Employer Organization Name'])
hourplace3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1166 entries, 0 to 1165
Data columns (total 64 columns):
 #   Column                                                       Non-Null Count  Dtype         
---  ------                                                       --------------  -----         
 0   Individual_x                                                 1166 non-null   object        
 1   Billing Date                                                 1157 non-null   datetime64[ns]
 2   Hours worked this MONTH                                      1157 non-null   float64       
 3   EmployerPlacementID                                          1165 non-null   float64       
 4   Content Type_x                                               1166 non-null   object        
 5   Created_x                                                    1166 non-null   datetime64[ns]
 6   Created By_x                                                 1166 non-null   object        
 7   Folder Child Co

In [238]:
hourplace4 = hourplace3[[
    'Individual2',
    'people_First Name',
    'Middle',
    'people_Last Name',
    'demo_Date of Birth',
    'Billing Date',
    'Employer Organization Name',
    'Placement Date',
    'employment Record ID',
    'Hours worked this MONTH',
    'Customized Employment: Wage'
]].copy()

hourplace4['associated employment'] = hourplace4['Employer Organization Name']+' Started: '+hourplace4['Placement Date'].astype(str)
hourplace5 = hourplace4[hourplace4['people_First Name'].notna()].copy()

In [239]:
hourplace6 = hourplace5.drop_duplicates(subset=['people_First Name','people_Last Name','Billing Date','employment Record ID',]).copy()

In [47]:
hourplace6.to_excel(r'redacted file path\exports\dda month reporting3.xlsx')

#### DDA Hours Assignment

In [240]:
service = pd.read_excel(r'redacted file path\Services - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
service[['Individual','Individual2']] = service['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
service = service.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
service['Individual2'] = service['Individual2'].str.replace(', ',',')

# remove test case rows
service = service[~service['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
service['Individual'] = service['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

service['Individual'] = service['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [241]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name',
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')
peepsmerge['demo_Date of Birth'] = peepsmerge['demo_Date of Birth'].fillna('1/1/1800')

In [242]:
servicemerge = service.merge(peepsmerge,how='left',left_on='Individual2',right_on='people_Full Name')

In [243]:
servicemerge['Service'] = servicemerge['Service'].str.replace('BFET - SL (Life Skills)','Life Skills')
servicemerge['Service'] = servicemerge['Service'].str.replace('DDA - Individual Employment Services','Individual Employment Services')
servicemerge['Service'] = servicemerge['Service'].str.replace('BFET - BR (Job Retention)','Job Retention')
servicemerge['Service'] = servicemerge['Service'].str.replace('DDA - Community Inclusion','Community Inclusion')
servicemerge['Service'] = servicemerge['Service'].str.replace('BFET - JS (Supervised Job Search)','Supervised Job Search')
servicemerge['Service'] = servicemerge['Service'].str.replace('BFET - JT (Job Search Training)','Job Search Training')
servicemerge['Service'] = servicemerge['Service'].str.replace('DDA - School to Work','School to Work')
servicemerge['Service'] = servicemerge['Service'].str.replace('DDA - Job Foundations','Job Foundations')
servicemerge['Service'] = servicemerge['Service'].str.replace('Staffing','Orion Staffing Assignment')
servicemerge['Service'].value_counts()

Service
Assessment                          629
Job Development                     580
Job Retention                       480
Community Based Assessment          211
Case Management                     143
Life Skills                         109
Training                            108
Individual Employment Services       89
Community Inclusion                  35
Intensive Training Services          34
Job Search Training                  26
Supervised Job Search                22
School to Work                       12
IL Services                           5
Job Foundations                       4
Internship - Other                    3
Trial Work Experience                 2
Pre-ETS IL Skills Training            1
Internship - CNC Mills                1
Internship - Mechanical Assembly      1
Name: count, dtype: int64

In [244]:
servicemerge['Service Category'] = np.where(servicemerge['Service'].str.contains('Assessment|Community Based Assessment|Trial Work Experience|Life Skills'), 'Assessment',
                             np.where(servicemerge['Service'].str.contains('Job Retention|Intensive Training Services|Off Site Psycho Social Supports - Non-Supported Employment|Off Site Psycho Social Supports - Supported Employment'), 'Job Retention',
                             np.where(servicemerge['Service'].str.contains('Supervised Job Search|Job Search Training|Job Development'), 'Job Development',
                             np.where(servicemerge['Service'].str.contains('Case Management'), 'Case Management',
                             np.where(servicemerge['Service'].str.contains('Community Inclusion|Individual Employment Services|Job Foundations|School to Work'), 'DDA',
                             np.where(servicemerge['Service'].str.contains('IL Services'), 'Independent Living',
                             np.where(servicemerge['Service'].str.contains('Internship = CNC Mills|Internship - Mechanical Assembly|Internship - Other|Training'), 'Training',
                             np.where(servicemerge['Service'].str.contains('Orion Staffing Assignment'), 'Staffing',
                             np.where(servicemerge['Service'].str.contains('Pre-ETS IL Peer Mentoring|Pre-ETS IL Self Advocacy|Pre-ETS IL Skills Training|Pre-ETS IL Work Related Systems Access|Pre-ETS Information Interview|Pre-ETS Job Exploration|Pre-ETS Job Shadow|Pre-ETS Social Skills|Pre-ETS WBL|Pre-ETS Workplace Readiness Training'), 'Pre ETS',
                             'Other')))))))))
servicemerge['Service Category'].value_counts()

Service Category
Assessment            1005
Job Development        628
Job Retention          514
Case Management        143
DDA                    140
Training               113
Independent Living       5
Other                    1
Name: count, dtype: int64

In [245]:
servicemerge['subservice'] = np.where(servicemerge['Service'].str.contains('Community Inclusion'), 'Community Inclusion',
                            np.where(servicemerge['Service'].str.contains('Individual Employment Services'), 'Individual Employment Services',
                            np.where(servicemerge['Service'].str.contains('School to Work'), 'School to Work',
                            np.where(servicemerge['Service'].str.contains('Job Development'), 'Job Development',
                            np.where(servicemerge['Service'].str.contains('Intensive Training Services'), 'Job Retention',
                            'Other')))))
servicemerge['subservice'].value_counts()

subservice
Other                             1745
Job Development                    580
Individual Employment Services      89
Community Inclusion                 89
Job Retention                       34
School to Work                      12
Name: count, dtype: int64

In [246]:
hours = pd.read_excel(r'redacted file path\exports\apricot dda hours assignment export.xlsx')

In [247]:
servicehourmerge = servicemerge.merge(hours,how='right',left_on=['people_First Name','people_Last Name','subservice'],right_on=['First','Last','Sub-Service Category'])
servicehourmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 37 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Start Date                            119 non-null    datetime64[ns]
 1   Individual                            119 non-null    object        
 2   Venue                                 119 non-null    object        
 3   End Date                              29 non-null     object        
 4   Service                               119 non-null    object        
 5   Recommendations                       0 non-null      object        
 6   Hours Allocated Per Month (DDA Only)  99 non-null     float64       
 7   Modified By                           119 non-null    object        
 8   Content Type                          119 non-null    object        
 9   Created                               119 non-null    datetime64[ns]
 10  Cr

In [248]:
servicehourmerge2 = servicehourmerge[[
    'First',
    'Middle',
    'Last',
    'Date of Birth',
    'Associated Service',
    'service Record ID',
    'Associated Sub Service',
    'Sub-Service Category',
    'Start Date',
    'End Date',
    'Hours Allocated Per Month (DDA Only)',
    'Assigned Programs'  
]].copy()

In [74]:
servicehourmerge2.to_excel(r'redacted file path\exports\real dda hours2.xlsx')

In [249]:
place = pd.read_excel(r'redacted file path\Employer placement hours - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
place[['Individual','Individual2']] = place['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
place = place.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
place['Individual2'] = place['Individual2'].str.replace(', ',',')

In [251]:
indprof = pd.read_csv(r'redacted file path\for import\IndividualProfile import1 reviewed.csv')
indprof = indprof.iloc[1:]

In [252]:
indprof['Individual2'] = indprof['field_3436_last']+','+indprof['field_3436_first']

In [253]:
placemerge = place.merge(indprof,how='left',on='Individual2')
placemerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 50 columns):
 #   Column                                                    Non-Null Count  Dtype         
---  ------                                                    --------------  -----         
 0   Individual                                                920 non-null    object        
 1   Billing Date                                              912 non-null    datetime64[ns]
 2   Hours worked this MONTH                                   912 non-null    float64       
 3   EmployerPlacementID                                       919 non-null    float64       
 4   Content Type                                              920 non-null    object        
 5   Created                                                   920 non-null    datetime64[ns]
 6   Created By                                                920 non-null    object        
 7   Folder Child Count                          

In [26]:
placemerge.to_excel(r'redacted file path\exports\end month report1.xlsx')

In [ ]:
comm = pd.read_excel(r'redacted file path\Client Community Activity - unformatted.xlsx')

Merge dda case info and individual profile data together

this was very messy so I just did it manually, hence the commented code

In [10]:
ddacase = pd.read_excel(r'redacted file path\DDA Case info - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
ddacase[['Individual','Individual2']] = ddacase['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
ddacase = ddacase.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
ddacase['Individual2'] = ddacase['Individual2'].str.replace(', ',',')

In [11]:
# people2 = pd.read_excel(r'redacted file path\exports\people merge2.xlsx')
# people3 = people2[['Last Name_x','First Name_x','Date of Birth','Full Name']].copy()

In [12]:
# ddacasemerge = ddacase.merge(people3,how='left',left_on='Individual2',right_on='Full Name')

,Title,Individual,Date,DDA Case File Number,Funding Code,Acuity Level,End Date,Date of Annual Plan,6 Month Review Due Date,Item Type,Path,Individual2,Last Name_x,First Name_x,Date of Birth,Full Name
0,View,1533,2023-10-26,773156.0,Basic Plus,Medium,NaT,NaT,1900-06-30,Item,Lists/DDACaseInfo,"Adams,Melissa",Adams,Melissa,1987-08-31 00:00:00,"Adams,Melissa"
1,View,1012,2020-05-11,772403.0,Basic Plus,Low,NaT,2023-06-01,2023-11-30,Item,Lists/DDACaseInfo,"Akers,Laura",Akers,Laura,1978-09-25 00:00:00,"Akers,Laura"
2,View,307,2016-09-28,872605.0,Basic Plus,High,NaT,2023-01-07,2023-07-08,Item,Lists/DDACaseInfo,"Almgren,Anders",Almgren,Anders,1981-10-31 00:00:00,"Almgren,Anders"


In [13]:
# ddacasemerge2 = ddacasemerge[['Last Name_x','First Name_x','Date of Birth','DDA Case File Number','Funding Code','Acuity Level']].copy()

In [14]:
# indprof = pd.read_csv(r'redacted file path\for import\IndividualProfile import1 unreviewed.csv')
# indprof = indprof.iloc[1:]

,System Header Row\nDO NOT DELETE THIS ROW\nOR THIS COLUMN!,field_3436_first,field_3436_middle,field_3436_last,field_3195,field_3848,field_2893_first,field_2893_middle,field_2893_last,field_3194,...,field_3212_country,field_3212_geolocation,field_3215,field_3216,field_3433,field_3434,field_3908,field_3909,field_3910,field_2899
1,NaN,Besharo,NaN,Abduwahab,8/30/2000,NaN,Besharo,NaN,Abduwahab,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Auburn
2,NaN,Noah,NaN,Abraham,4/2/1972,NaN,Noah,NaN,Abraham,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Auburn
3,NaN,Rebecca,NaN,Acosta,5/1/1979,NaN,Rebecca,NaN,Acosta,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,NaN
4,NaN,Jnean,NaN,Acteson,5/22/1980,NaN,Jnean,NaN,Acteson,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Auburn
5,NaN,Chris,NaN,Adair,11/26/1995,NaN,Chris,NaN,Adair,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,NaN,Janet,NaN,Zello,3/29/1993,NaN,Janet,NaN,Zello,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,NaN
1408,NaN,Tevin,NaN,Zeno,2/21/1991,NaN,Tevin,NaN,Zeno,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Auburn
1409,NaN,Manuel,NaN,Zepeda,9/6/1980,NaN,Manuel,NaN,Zepeda,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Auburn
1410,NaN,Edwin,NaN,Zhang,10/30/1998,NaN,Edwin,NaN,Zhang,NaN,...,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,Mukilteo


In [15]:
# indprofmerge = indprof.merge(ddacasemerge2,how='left',
#                              left_on=['field_3436_first','field_3436_last','field_3195'],
#                             right_on=['First Name_x','Last Name_x','Date of Birth'])
# indprofmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 39 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   System Header Row
DO NOT DELETE THIS ROW
OR THIS COLUMN!  0 non-null      object 
 1   field_3436_first                                          1411 non-null   object 
 2   field_3436_middle                                         0 non-null      object 
 3   field_3436_last                                           1411 non-null   object 
 4   field_3195                                                1215 non-null   object 
 5   field_3848                                                0 non-null      object 
 6   field_2893_first                                          1411 non-null   object 
 7   field_2893_middle                                         0 non-null      object 
 8   field_2893_last   

In [16]:
# remove = indprofmerge[indprofmerge['Last Name_x'].notna()]
# remove

,System Header Row\nDO NOT DELETE THIS ROW\nOR THIS COLUMN!,field_3436_first,field_3436_middle,field_3436_last,field_3195,field_3848,field_2893_first,field_2893_middle,field_2893_last,field_3194,...,field_3908,field_3909,field_3910,field_2899,Last Name_x,First Name_x,Date of Birth,DDA Case File Number,Funding Code,Acuity Level
78,NaN,Dominick,NaN,Bailey,NaN,NaN,Dominick,NaN,Bailey,NaN,...,NaN,NaN,NaN,Mukilteo,Bailey,Dominick,NaN,445246.0,Basic Plus,Medium
79,NaN,Dominick,NaN,Bailey,NaN,NaN,Dominick,NaN,Bailey,NaN,...,NaN,NaN,NaN,Mukilteo,Bailey,Dominick,NaN,445246.0,Basic Plus,Medium
171,NaN,Michael,NaN,Bryant,NaN,NaN,Michael,NaN,Bryant,NaN,...,NaN,NaN,NaN,Mukilteo,Bryant,Michael,NaN,750285.0,Core,NaN
485,NaN,Annika,NaN,Hagemeier,NaN,NaN,Annika,NaN,Hagemeier,NaN,...,NaN,NaN,NaN,Mukilteo,Hagemeier,Annika,NaN,876318.0,School to Work,NaN
1017,NaN,Tatum,NaN,Rehm,NaN,NaN,Tatum,NaN,Rehm,NaN,...,NaN,NaN,NaN,Mukilteo,Rehm,Tatum,NaN,NaN,School to Work,NaN
1040,NaN,Dennis,NaN,Rockwell,NaN,NaN,Dennis,NaN,Rockwell,NaN,...,NaN,NaN,NaN,Mukilteo,Rockwell,Dennis,NaN,884390.0,Fircrest,Medium
1199,NaN,Desiree,NaN,Stuard,NaN,NaN,Desiree,NaN,Stuard,NaN,...,NaN,NaN,NaN,Mukilteo,Stuard,Desiree,NaN,864046.0,Basic Plus,Medium
1200,NaN,Desiree,NaN,Stuard,NaN,NaN,Desiree,NaN,Stuard,NaN,...,NaN,NaN,NaN,Mukilteo,Stuard,Desiree,NaN,864046.0,Basic Plus,Medium


In [17]:
# ddacasemerge3 = pd.merge(ddacasemerge2,remove,how='outer',
#                          left_on=['First Name_x','Last Name_x','Date of Birth'],
#                          right_on=['field_3436_first','field_3436_last','field_3195'])

In [18]:
# indprofmerge.to_excel(r'redacted file path\exports\ind.xlsx')
# remove.to_excel(r'redacted file path\exports\dda remove.xlsx')
# ddacasemerge2.to_excel(r'redacted file path\exports\dda ind.xlsx')

In [51]:
indprof['field_3848']

,Client Alerts
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1406,NaN
1407,NaN
1408,NaN
1409,NaN


### Service funding staff

In [254]:
fund = pd.read_excel(r'redacted file path\Funding - unformatted.xlsx')

# split out the contents of the client column on the ;# and add the split values to the newly created individual column
fund[['Individual','Individual2']] = fund['Individual'].str.split(';#', expand=True)

# sort the individual column values alphabetically
fund = fund.sort_values('Individual2',ascending=True)

# the key value had a space in it, so needed to remove that space
fund['Individual2'] = fund['Individual2'].str.replace(', ',',')

# remove test case rows
fund = fund[~fund['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
fund['Individual'] = fund['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

fund['Individual'] = fund['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [255]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')

In [256]:
fund2 = fund.merge(peepsmerge, how='left',left_on='Individual2',right_on='people_Full Name')
fund2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1318 entries, 0 to 1317
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Individual                       1318 non-null   object        
 1   Venue                            1308 non-null   object        
 2   Service                          1152 non-null   object        
 3   Funding Source                   1295 non-null   object        
 4   Start Date                       1316 non-null   datetime64[ns]
 5   End Date                         1073 non-null   datetime64[ns]
 6   Final Funding Amount             466 non-null    float64       
 7   Projected End / Billing Date     742 non-null    datetime64[ns]
 8   Projected Funding Amount         473 non-null    float64       
 9   Staff                            1180 non-null   object        
 10  Modified                         1318 non-null   datetime64[

In [257]:
fund2 = fund2.drop(columns=['Title','Item Type','Path','Optional DVR Billing Tracking!!'])

In [258]:
fund2['Service'].value_counts()

Service
Job Development                                               208
Assessment                                                    197
Community Based Assessment                                    158
BFET - SL (Life Skills)                                       118
Job Retention                                                  89
Case Management                                                84
DDA - Individual Employment Services                           77
Training                                                       57
Intensive Training Services                                    36
DDA - Community Inclusion                                      35
BFET - BR (Job Retention)                                      32
BFET - JS (Supervised Job Search)                              25
BFET - JT (Job Search Training)                                16
DDA - School to Work                                           11
IL Services                                                     2
Of

In [259]:
# edit service names

fund2['Service'] = fund2['Service'].str.replace('BFET - SL (Life Skills)','Life Skills')
fund2['Service'] = fund2['Service'].str.replace('DDA - Individual Employment Services','Individual Employment Services')
fund2['Service'] = fund2['Service'].str.replace('BFET - BR (Job Retention)','Job Retention')
fund2['Service'] = fund2['Service'].str.replace('DDA - Community Inclusion','Community Inclusion')
fund2['Service'] = fund2['Service'].str.replace('BFET - JS (Supervised Job Search)','Supervised Job Search')
fund2['Service'] = fund2['Service'].str.replace('BFET - JT (Job Search Training)','Job Search Training')
fund2['Service'] = fund2['Service'].str.replace('DDA - School to Work','School to Work')
fund2['Service'] = fund2['Service'].str.replace('DDA - Job Foundations','Job Foundations')
fund2['Service'] = fund2['Service'].str.replace('Staffing','Orion Staffing Assignment')

In [260]:
# add a service category to the df

fund2['Service Category'] = np.where(fund2['Service'].str.contains('Assessment|Community Based Assessment|Trial Work Experience|Life Skills'), 'Assessment',
                             np.where(fund2['Service'].str.contains('Job Retention|Intensive Training Services|Off Site Psycho Social Supports - Non-Supported Employment|Off Site Psycho Social Supports - Supported Employment'), 'Job Retention',
                             np.where(fund2['Service'].str.contains('Supervised Job Search|Job Search Training|Job Development'), 'Job Development',
                             np.where(fund2['Service'].str.contains('Case Management'), 'Case Management',
                             np.where(fund2['Service'].str.contains('Community Inclusion|Individual Employment Services|Job Foundations|School to Work'), 'DDA',
                             np.where(fund2['Service'].str.contains('IL Services'), 'Independent Living',
                             np.where(fund2['Service'].str.contains('Internship = CNC Mills|Internship - Mechanical Assembly|Internship - Other|Training'), 'Training',
                             np.where(fund2['Service'].str.contains('Orion Staffing Assignment'), 'Staffing',
                             np.where(fund2['Service'].str.contains('Pre-ETS IL Peer Mentoring|Pre-ETS IL Self Advocacy|Pre-ETS IL Skills Training|Pre-ETS IL Work Related Systems Access|Pre-ETS Information Interview|Pre-ETS Job Exploration|Pre-ETS Job Shadow|Pre-ETS Social Skills|Pre-ETS WBL|Pre-ETS Workplace Readiness Training'), 'Pre ETS',
                             'Other')))))))))

In [261]:
fund2['location code'] = np.where(fund2['Venue'].str.contains('Auburn'),1174,2870)

In [262]:
fund2.columns

Index(['Individual', 'Venue', 'Service', 'Funding Source', 'Start Date',
       'End Date', 'Final Funding Amount', 'Projected End / Billing Date',
       'Projected Funding Amount', 'Staff', 'Modified', 'Date Billed - Intake',
       'Amount Billed - Intake', 'Date Billed - Activity',
       'Amount Billed - Activity', 'Date Billed - Outcome',
       'Amount Billed - Outcome', 'Billing - Bonus', 'Content Type', 'Created',
       'Created By', 'Folder Child Count', 'ID', 'Item Child Count',
       'Modified By', 'Type of Funding', 'Individual2', 'people_First Name',
       'people_Last Name', 'demo_Date of Birth', 'people_Full Name',
       'Service Category', 'location code'],
      dtype='object')

In [263]:
fund3 =  fund2[fund2['people_Full Name'].notna()].copy()
fund3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 775 entries, 5 to 1317
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Individual                    775 non-null    object        
 1   Venue                         771 non-null    object        
 2   Service                       694 non-null    object        
 3   Funding Source                765 non-null    object        
 4   Start Date                    775 non-null    datetime64[ns]
 5   End Date                      582 non-null    datetime64[ns]
 6   Final Funding Amount          181 non-null    float64       
 7   Projected End / Billing Date  411 non-null    datetime64[ns]
 8   Projected Funding Amount      206 non-null    float64       
 9   Staff                         689 non-null    object        
 10  Modified                      775 non-null    datetime64[ns]
 11  Date Billed - Intake          35 non

In [264]:
fund3 = fund3[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Start Date',
    'End Date',
    'Service Category',
    'Service',
    'Funding Source',
    'Staff',
    'location code'
]].copy()

In [265]:
# create a migration id column

fund3['migrationID'] = range(1,len(fund3)+1)

In [266]:
fund3['Funding Source'] = fund3['Funding Source'].fillna('Unfunded')
# fund3['DVR Office'] = fund3[fund3['Funding Source'].str.contains('DVR')] 
# fund3

# Create a new column with values containing "DVR"
fund3['DVR Office'] = fund3['Funding Source'].where(fund3['Funding Source'].str.contains('DVR'), np.nan)

# Remove text following "DVR" in the original column
fund3['Funding Source'] = fund3['Funding Source'].str.replace(r'DVR.*', 'DVR', regex=True)

# Do same thing that was done with DVR but with DDA
fund3['DDA Office'] = fund3['Funding Source'].where(fund3['Funding Source'].str.contains('DDA'), np.nan)

fund3['Funding Source'] = fund3['Funding Source'].str.replace(r'DDA.*', 'DDA', regex=True)

In [267]:
fund3['Funding Source'].value_counts()

Funding Source
DVR                                                                 290
Snohomish County Human Services | Housing and Community Services    130
DDA                                                                 110
BFET                                                                 94
King County Jobs and Housing                                         69
City of Auburn                                                       29
City of Federal Way                                                  19
WRT - State Board for Community and Technical Colleges               12
Unfunded                                                             10
City of Renton                                                        4
City of Kent                                                          3
SJI (Seattle Jobs Initiative)                                         2
DSB-Tacoma                                                            1
Labor & Industries                               

In [268]:
fundrep = {
    'Olsen & Associates':'Labor & Industries',
    'Strategic Consulting Services':'Labor & Industries',
    'Pioneer Industries':'Labor & Industries',
    'Aerospace Joint Apprenticeship Committee (AJAC)':'Labor & Industries',
    'Workforce Training and Education Coordinating Board':'WRT',
    'WRT - State Board for Community and Technical Colleges':'WRT',
    'City of Renton':'City Funding - Human Services (Auburn, Renton)',
    'City of Kent':'City Funding - Human Services (Auburn, Renton)',
    'City of Auburn':'City Funding - Human Services (Auburn, Renton)',
    'City of Federal Way':'City Funding - CDBG (Federal Way)',
    'Snohomish County Human Services | Housing and Community Services':'Snohomish County - ERSS',
    'DDA-King County':'DDA',
    'DDA-Snohomish':'DDA',
    'Veterans Administration-Seattle':'Veterans Administration',
    'King County Jobs and Housing':'King County Jobs & Housing',
    'SJI (Seattle Jobs Initiative)':'Labor & Industries',
    'DSB-Tacoma':'DVR'
}

fund3['Funding Source'] = fund3['Funding Source'].replace(fundrep)

In [469]:
fund3.to_excel(r'redacted file path\exports\real service funding staff.xlsx')

#### Program

In [269]:
service = pd.read_excel(r'redacted file path\Services - unformatted.xlsx')

# remove repetitive columns
service = service.drop(columns=['Title','Item Type','Path','Folder Child Count','Created By','Content Type','Item Child Count'])

# remove the (###) from the individual name
service['Individual'] = service['Individual'].replace(r'\b\d{1,4};#','',regex=True)

# the key value had a space in it, so needed to remove that space
service['Individual'] = service['Individual'].str.replace(', ',',')

# remove test case rows
service = service[~service['Individual'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
service['Individual'] = service['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

service['Individual'] = service['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

In [270]:
venue = pd.read_excel(r'redacted file path\venue history - unformatted.xlsx')

# remove repetitive columns
venue = venue.drop(columns=['Item Type','Path','Folder Child Count','Created By','Content Type','Item Child Count'])

# the key value had a space in it, so needed to remove that space
venue['Individual'] = venue['Individual'].str.replace(', ',',')

# remove test case rows
venue = venue[~venue['Individual'].str.contains('test', case=False, na=False)].reset_index().copy()

# remove any text (within parethesis) to standardize names
venue['Individual'] = venue['Individual'].replace(r'\(.*?\)','',regex=True).str.strip()

venue['Individual'] = venue['Individual'].replace(r'\".*?\"','',regex=True).str.strip()

venue2 = venue.add_prefix('venue_')

In [271]:
venue2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   venue_index                2208 non-null   int64         
 1   venue_Individual           2208 non-null   object        
 2   venue_Venue                2208 non-null   object        
 3   venue_Created              2208 non-null   datetime64[ns]
 4   venue_Start Date           2208 non-null   datetime64[ns]
 5   venue_Exit Date            1995 non-null   datetime64[ns]
 6   venue_Status               2208 non-null   object        
 7   venue_Referred By          832 non-null    object        
 8   venue_Shift                1165 non-null   object        
 9   venue_Shift 2 Change Date  33 non-null     datetime64[ns]
 10  venue_Shift Notes          351 non-null    object        
 11  venue_Orion Staff          0 non-null      float64       
 12  venue_

In [272]:
peepsmerge = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'people_Full Name'
]].copy()

peepsmerge['people_Full Name'] = peepsmerge['people_Full Name'].str.replace(', ',',')

In [273]:
peepsmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   people_First Name   528 non-null    object
 1   people_Last Name    528 non-null    object
 2   demo_Date of Birth  489 non-null    object
 3   people_Full Name    528 non-null    object
dtypes: object(4)
memory usage: 16.6+ KB


In [274]:
# service2 = service.merge(peepsmerge, how='left',left_on='Individual',right_on='people_Full Name')
# service2.info()

In [275]:
venue3 = venue2.merge(peepsmerge, how='left',left_on='venue_Individual',right_on='people_Full Name')

venue3['location code'] = np.where(venue3['venue_Venue'].str.contains('Auburn'),1174,2870)
venue3['location'] = np.where(venue3['venue_Venue'].str.contains('Auburn'),'Auburn','Mukilteo')

venue3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2216 entries, 0 to 2215
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   venue_index                2216 non-null   int64         
 1   venue_Individual           2216 non-null   object        
 2   venue_Venue                2216 non-null   object        
 3   venue_Created              2216 non-null   datetime64[ns]
 4   venue_Start Date           2216 non-null   datetime64[ns]
 5   venue_Exit Date            2002 non-null   datetime64[ns]
 6   venue_Status               2216 non-null   object        
 7   venue_Referred By          835 non-null    object        
 8   venue_Shift                1169 non-null   object        
 9   venue_Shift 2 Change Date  33 non-null     datetime64[ns]
 10  venue_Shift Notes          353 non-null    object        
 11  venue_Orion Staff          0 non-null      float64       
 12  venue_

In [276]:
# service2['location code'] = np.where(service2['Venue'].str.contains('Auburn'),1174,2870)
# service2['location'] = np.where(service2['Venue'].str.contains('Auburn'),'Auburn','Mukilteo')

In [277]:
# service2['Venue'].value_counts()

In [278]:
# change the venue to match the training program

# service2['Venue'] = service2['Venue'].str.replace('Auburn - Office Skills & Customer Service','Clerical Customer Service Training')
# service2['Venue'] = service2['Venue'].str.replace('Auburn - Contact Center','Clerical Customer Service Training')
# service2['Venue'] = service2['Venue'].str.replace('Auburn - Manufacturing Program','Transitional Training - Manufacturing')
# service2['Venue'] = service2['Venue'].str.replace('Mukilteo - Manufacturing Program','Transitional Training - Manufacturing')
# service2['Venue'] = service2['Venue'].str.replace('Auburn - Custodial','Transitional Training - Custodial')
# service2['Venue'] = service2['Venue'].str.replace('Mukilteo-Light Assembly','Transitional Training - Manufacturing')
# ### service2['Venue'] = service2['Venue'].str.replace('Auburn - Community Based Services','')
# ### service2['Venue'] = service2['Venue'].str.replace('Mukilteo - Community Based Services','')


In [279]:
# change the venue to match the training program

venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Auburn - Office Skills & Customer Service','Clerical Customer Service Training')
venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Auburn - Contact Center','Clerical Customer Service Training')
venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Auburn - Manufacturing Program','Transitional Training - Manufacturing')
venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Mukilteo - Manufacturing Program','Transitional Training - Manufacturing')
venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Auburn - Custodial','Transitional Training - Custodial')
venue3['venue_Venue'] = venue3['venue_Venue'].str.replace('Mukilteo-Light Assembly','Transitional Training - Manufacturing')
### venue3['Venue'] = venue3['Venue'].str.replace('Auburn - Community Based Services','')
### venue3['Venue'] = venue3['Venue'].str.replace('Mukilteo - Community Based Services','')

In [280]:
venue3.columns

Index(['venue_index', 'venue_Individual', 'venue_Venue', 'venue_Created',
       'venue_Start Date', 'venue_Exit Date', 'venue_Status',
       'venue_Referred By', 'venue_Shift', 'venue_Shift 2 Change Date',
       'venue_Shift Notes', 'venue_Orion Staff', 'venue_Assigned Staff',
       'venue_Modified', 'venue_Eligible?', 'venue_Exit Reason',
       'venue_Header -Shift2', 'venue_Header-Shift', 'venue_ID',
       'venue_Modified By', 'venue_Notes', 'venue_Referral Date',
       'venue_Shift 2', 'venue_Shift 2 Notes', 'venue_Shift 3',
       'venue_Shift 3 Change Date', 'venue_Shift 3 Notes', 'venue_View',
       'people_First Name', 'people_Last Name', 'demo_Date of Birth',
       'people_Full Name', 'location code', 'location'],
      dtype='object')

In [281]:
venue4 =  venue3[venue3['people_Full Name'].notna()].copy()
venue4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 882 entries, 20 to 2215
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   venue_index                882 non-null    int64         
 1   venue_Individual           882 non-null    object        
 2   venue_Venue                882 non-null    object        
 3   venue_Created              882 non-null    datetime64[ns]
 4   venue_Start Date           882 non-null    datetime64[ns]
 5   venue_Exit Date            671 non-null    datetime64[ns]
 6   venue_Status               882 non-null    object        
 7   venue_Referred By          353 non-null    object        
 8   venue_Shift                456 non-null    object        
 9   venue_Shift 2 Change Date  8 non-null      datetime64[ns]
 10  venue_Shift Notes          242 non-null    object        
 11  venue_Orion Staff          0 non-null      float64       
 12  venue_Assig

In [282]:
venue4 = venue4[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth', 
    'venue_Venue',
    'location',
    'venue_Start Date',
    'venue_Exit Date',
    'venue_Exit Reason',
    'venue_Notes',
    'venue_Shift',
    'venue_Shift Notes',
    'location code'
]].copy()

venue4['Status'] = np.where(venue4['venue_Exit Date'].isna(),'Program Started','Closed No Program Services')

In [283]:
venue5 = venue4[~venue4['venue_Venue'].str.contains('Community Based Services')].reset_index().copy()

In [284]:
venue5['venue_Venue'].value_counts()

venue_Venue
Transitional Training - Manufacturing    361
Clerical Customer Service Training        41
Transitional Training - Custodial          8
Name: count, dtype: int64

In [285]:
venue6 = venue5[~venue5['venue_Venue'].str.contains('test', case=False, na=False)].copy()

venue6['venue_Venue'].value_counts()

venue_Venue
Transitional Training - Manufacturing    361
Clerical Customer Service Training        41
Transitional Training - Custodial          8
Name: count, dtype: int64

In [286]:
venue6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   index               410 non-null    int64         
 1   people_First Name   410 non-null    object        
 2   people_Last Name    410 non-null    object        
 3   demo_Date of Birth  397 non-null    object        
 4   venue_Venue         410 non-null    object        
 5   location            410 non-null    object        
 6   venue_Start Date    410 non-null    datetime64[ns]
 7   venue_Exit Date     349 non-null    datetime64[ns]
 8   venue_Exit Reason   331 non-null    object        
 9   venue_Notes         191 non-null    object        
 10  venue_Shift         392 non-null    object        
 11  venue_Shift Notes   238 non-null    object        
 12  location code       410 non-null    int32         
 13  Status              410 non-null    object        

In [452]:
venue6.to_excel(r'redacted file path\exports\real enrollment to program.xlsx')

#### Service

In [287]:
enroll = merge6[[
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'intake_Orion Location',
    'intake_Date of Intake',
    'intake_Intake Admin Notes',
    'intake_Status of Intake',
    'intake_Start Date',
    'intake_Processing details: Funding and Project Eligibility',
    'intake_Assigned to Venue',
    'intake_Processing details: Criminal History Check',
    'intake_Processing details: Date Closed',
    'intake_Processing details: Closed'
]].copy()

enroll['location code'] = enroll['intake_Orion Location']
enroll['location code'] = enroll['location code'].replace('Auburn',1174)
enroll['location code'] = enroll['location code'].replace('Mukilteo',2870)

enrollcolname = [
    'people_First Name',
    'people_Last Name',
    'demo_Date of Birth',
    'Location',
    'Date intake started',
    'Service intake processing notes',
    'Status',
    'Date services starting',
    'Eligible funders',
    'Is client also participating in a training program?',
    'Criminal History Check',
    'Date Closing',
    'Closing Note',
    'Assigned programs'
]

enroll.columns = enrollcolname

In [288]:
# replace values in status

statrep = {
    'Started':'Services Started',
    'Withdrawn':'Closed No Services',
    'Denied':'Closed No Services',
    'Approved':'On Hold',
    'Tour Only':'Intake Processing',
    
}
enroll['Status'] = enroll['Status'].replace(statrep)

In [289]:
import re

enroll.loc[:,'Eligible funders'] = enroll['Eligible funders'].replace(r';#\d{1,2}','|',regex=True)
enroll.loc[:,'Eligible funders'] = enroll['Eligible funders'].str.rstrip('|')
enroll.loc[:,'Eligible funders'] = enroll['Eligible funders'].str.replace(';#','')
enroll['Eligible funders'].value_counts()

Eligible funders
Snohomish County CSBG                                          68
BFET                                                           43
King County Jobs & Housing                                     37
Auburn Human Services Grant                                    15
Federal Way CDBG Program                                       10
BFET|Auburn Human Services Grant                                8
Renton Human Services Grant                                     6
BFET|King County Jobs & Housing                                 5
Auburn Human Services Grant|BFET                                5
Auburn Human Services Grant|BFET|King County Jobs & Housing     5
SBCTC - WRT Program                                             4
King County Jobs & Housing|Auburn Human Services Grant          3
Auburn Human Services Grant|King County Jobs & Housing|BFET     3
REDF                                                            3
Auburn Human Services Grant|King County Jobs & Housing     

In [290]:
# find unique values in the eligible funders column

unique_funders = enroll['Eligible funders'].unique()

split_items = [item for sublist in unique_funders if isinstance(sublist, str) for item in sublist.split('|')]
uniquedf = pd.DataFrame(split_items)
unique_items = uniquedf[0].unique()

In [291]:
for i in unique_items:
    print(i)

Snohomish County CSBG
Federal Way CDBG Program
King County Jobs & Housing
Auburn Human Services Grant
BFET
SBCTC - WRT Program
Renton Human Services Grant
REDF
RTI Study
Auburn COVID Grant
CareerReady (SJI)
Kent Human Services Grant
Kent COVID Grant


In [292]:
elrep = {
    'Renton Human Services Grant':'City of Renton',
    'Auburn Human Services Grant':'City of Auburn',
    'Kent Human Services Grant':'City of Kent',
    'Auburn COVID Grant':'City of Auburn',
    'Kent COVID Grant':'City of Kent',
    'Federal Way CDBG Program':'City of Federal Way',
    'King County Jobs & Housing':'King County Jobs and Housing',
    'Snohomish County CSBG':'Snohomish County Human Services Housing and Community Services',
    'SBCTC - WRT Program':'WRT - State Board for Community and Technical Colleges',
    'REDF':'',
    'CareerReady \(SJI\)':'',
    'RTI Study':'',
    'WDC - Youth Internship':''
}

enroll['Eligible funders'] = enroll['Eligible funders'].replace(elrep,regex=True)
enroll['Eligible funders'] = enroll['Eligible funders'].str.replace('||','|')
enroll['Eligible funders'] = enroll['Eligible funders'].str.replace('||','|')
enroll['Eligible funders'] = enroll['Eligible funders'].str.strip('|')

In [293]:
enroll['Eligible funders'].value_counts()

Eligible funders
Snohomish County Human Services Housing and Community Services                68
BFET                                                                          43
King County Jobs and Housing                                                  37
City of Auburn                                                                19
City of Federal Way                                                           11
BFET|City of Auburn                                                            8
City of Auburn|BFET                                                            6
City of Renton                                                                 6
City of Auburn|BFET|King County Jobs and Housing                               5
BFET|King County Jobs and Housing                                              5
WRT - State Board for Community and Technical Colleges                         5
                                                                               4
City of Aub

In [294]:
# set training program to binary response

enroll['Is client also participating in a training program?'] = np.where(enroll['Is client also participating in a training program?'].str.contains('Manufacturing Program|Office Skills & Customer Service|Custodial|Contact Center'),'Yes','No')

In [295]:
enroll['Is client also participating in a training program?'].value_counts()

Is client also participating in a training program?
Yes    396
No     132
Name: count, dtype: int64

In [296]:
enroll['National Criminal History Check'] = np.where(enroll['Criminal History Check'] == 'National - pending','Pending','')
enroll['Criminal History Check'] = enroll['Criminal History Check'].str.replace('National - pending','')

In [421]:
enroll.to_excel(r'redacted file path\exports\real enrollment to service import2.xlsx')

### Intake

In [297]:
intake = pd.read_excel(r'redacted file path\Intake Wizard (intake application) - unformatted.xlsx')

# remove repetitive columns
intake = intake.drop(columns=['Title','Item Type','Path','Folder Child Count','Content Type','Item Child Count'])

# the key value had a space in it, so needed to remove that space
intake['Full Name'] = intake['Full Name'].str.replace(', ',',')

# remove test case rows
intake = intake[~intake['Full Name'].str.contains('test', case=False, na=False)].copy()

# remove any text (within parethesis) to standardize names
intake['Full Name'] = intake['Full Name'].replace(r'\(.*?\)','',regex=True).str.strip()

intake['Full Name'] = intake['Full Name'].replace(r'\".*?\"','',regex=True).str.strip()

In [298]:
intake2 = intake[[
    'Created By',
    'Created',
    'Processing details: Closed',
    'Processing details: Date Closed',
    'Processing details: Closed',
    'Processing details: notes',
    'Created',
    'Orion Location',
    'Assigned to Venue',
    'Assigned to Venue',
    'Referral details: Referral Source',
    'Date of Tour',
    'Date of Tour',
    'Referral details: Referral Source',
    'Referral Contact:First Name',
    'Referral Contact:Last Name',
    'First Name',
    'Last Name',
    'Nickname',
    'Phone',
    'Phone',
    'Email',
    'Address',
    'City',
    'State',
    'Zip Code:County',
    'Zip Code',
    'Guardian: Status',
    'Guardian: email',
    'Guardian: First Name',
    'Guardian: Last Name',
    'Guardian: Phone',
    'Status of Intake',
    'Orion Location'
]]
colname = [
    'Assigned Staff',
    'Date Assigned',
    'Closing Referral without Moving Forward?',
    'Date Closed',
    'Referral Closed Reason',
    'Processing Notes',
    'Date',
    'Site',
    'Auburn Programs/Services Referring To',
    'Mukilteo Programs/Services Referring To',
    'Referral Source',
    'Is this Application being completed on site at Orion as part of a Site Tour?',
    'Date of Tour',
    'Agency Name',
    'Referral Contact:First Name',
    'Referral Contact:Last Name',
    'First Name',
    'Last Name',
    'Nickname',
    'Phone availabile',
    'Phone',
    'Email',
    'Address',
    'City',
    'State',
    'Zip Code:County',
    'Zip Code',
    'Guardian: Status',
    'Guardian: email',
    'Guardian: First Name',
    'Guardian: Last Name',
    'Guardian: Phone',
    'Status',
    'Orion Location'
]
intake2.columns = colname

In [299]:
# edit the duplicated columns with static values
intake2 = intake2.copy()
intake2.loc[intake2['Closing Referral without Moving Forward?'].notna(), 'Closing Referral without Moving Forward?'] = 'Yes'

#set the date value as a str
intake2['Is this Application being completed on site at Orion as part of a Site Tour?'] = intake2['Is this Application being completed on site at Orion as part of a Site Tour?'].astype(str)
intake2.loc[intake2['Is this Application being completed on site at Orion as part of a Site Tour?'].notna(), 'Is this Application being completed on site at Orion as part of a Site Tour?'] = 'Yes'

intake2.loc[intake2['Phone availabile'].notna(), 'Phone availabile'] = 'Yes'

In [300]:

intake2.loc[:,'Guardian info'] = intake2['Guardian: email']+' '+intake2['Guardian: First Name']+' '+intake2['Guardian: Last Name']+' '+intake2['Guardian: Phone']

intake3 = intake2.drop(columns=['Guardian: email','Guardian: First Name','Guardian: Last Name','Guardian: Phone']).copy()
intake3['Orion Location'] = intake3['Orion Location'].replace('Auburn',1174)
intake3['Orion Location'] = intake3['Orion Location'].replace('Mukilteo',2870)

In [301]:
# remove values from the site specific service

auburn_mask = intake3['Site'] == 'Auburn'
mukilteo_mask = intake3['Site'] == 'Mukilteo'

# Apply masks to set values to NaN where they don't match the site
intake3.loc[auburn_mask, 'Mukilteo Programs/Services Referring To'] = np.nan
intake3.loc[mukilteo_mask, 'Auburn Programs/Services Referring To'] = np.nan

In [302]:
intake3['Mukilteo Programs/Services Referring To'].value_counts()

Mukilteo Programs/Services Referring To
Mukilteo - Community Based Services    188
Mukilteo - Manufacturing Program        94
Mukilteo-Light Assembly                  1
Name: count, dtype: int64

In [303]:
intake3['Auburn Programs/Services Referring To'].value_counts()

Auburn Programs/Services Referring To
Auburn - Manufacturing Program               716
Auburn - Office Skills & Customer Service    103
Auburn - Community Based Services             32
Auburn - Custodial                            22
Auburn - Contact Center                        2
Mukilteo - Manufacturing Program               1
Name: count, dtype: int64

In [304]:
# Services referring to edit

mukdict = {
    'Mukilteo - Community Based Services':'Community Based Services',
    'Mukilteo - Manufacturing Program':'Transitional Training - Manufacturing',
    'Mukilteo-Light Assembly':'Transitional Training - Manufacturing'
}
aubdict = {
    'Auburn - Manufacturing Program':'Transitional Training - Manufacturing',
    'Auburn - Office Skills & Customer Service':'Clerical Customer Service Training',
    'Auburn - Community Based Services':'Community Based Services',
    'Auburn - Custodial':'Transitional Training - Custodial',
    'Auburn - Contact Center':'Clerical Customer Service Training',
    'Mukilteo - Manufacturing Program':'Transitional Training - Manufacturing'
}

intake3['Mukilteo Programs/Services Referring To'] = intake3['Mukilteo Programs/Services Referring To'].replace(mukdict)
intake3['Auburn Programs/Services Referring To'] = intake3['Auburn Programs/Services Referring To'].replace(aubdict)

In [305]:
# processing notes edit

refrep = {
    'Denied':'Client not a good fit',
    'Requested to Close':'Client refused services',
    'Timed Out':'Client unable to be reached',
}

intake3['Referral Closed Reason'] = intake3['Referral Closed Reason'].replace(refrep)

In [306]:
intake3['Referral Source'].value_counts()

Referral Source
75;#Previous Program Participant or Word of Mouth    237
73;#WorkSource                                       100
76;#AAHAA                                             96
45;#DVR-Lynnwood                                      90
35;#DVR-Seatac                                        67
                                                    ... 
86;#Olsen & Associates                                 1
77;#Goodwill                                           1
87;#Strategic Consulting Services                      1
63;#Tacoma Rescue Mission                              1
136;#Senior Housing Stability Program                  1
Name: count, Length: 65, dtype: int64

In [307]:
# referral source edit values

pattern = r'\b\d{1,3};#'
intake3['Referral Source'] = intake3['Referral Source'].str.replace(pattern,'',regex=True)

# Remove text following "DVR" in the original column
intake3['Referral Source'] = intake3['Referral Source'].str.replace(r'DVR.*', 'DVR', regex=True)

intake3['Referral Source'] = intake3['Referral Source'].str.replace(r'DDA.*', 'DDA', regex=True)

ref2rep = {
    'Previous Program Participant or Word of Mouth':'Family/Friend',
    'WorkSource':'Private Referral Partner (example, L&I)',
    'AAHAA':'Private Referral Partner (example, L&I)',
    'Senior Housing Stability Program':'Private Referral Partner (example, L&I)',
    'Tacoma Rescue Mission':'Private Referral Partner (example, L&I)',
    'Strategic Consulting Services':'Private Referral Partner (example, L&I)',
    'Goodwill':'Private Referral Partner (example, L&I)',
    'Olsen & Associates':'Private Referral Partner (example, L&I)',
    'Fusion Family Center':'Private Referral Partner (example, L&I)',
    'A Walk to Freedom Counseling':'Private Referral Partner (example, L&I)',
    'Peer Kent':'Private Referral Partner (example, L&I)',
    'Renton Community Court Resource Center':'Other Government Organization',
    'YWCA':'Private Referral Partner (example, L&I)',
    'Asian Counseling and Referral Service (ACRS)':'Private Referral Partner (example, L&I)',
    'Union Gospel Mission':'Private Referral Partner (example, L&I)',
    'Freedom Project':'Private Referral Partner (example, L&I)',
    'Compass Career Solutions':'Private Referral Partner (example, L&I)',
    'Aerospace Joint Apprenticeship Committee (AJAC)':'Private Referral Partner (example, L&I)',
    'Auburn Food Bank':'Private Referral Partner (example, L&I)',
    'Labor & Industries':'Private Referral Partner (example, L&I)',
    'Valley Cities':'Private Referral Partner (example, L&I)',
    'Sound Mental Health':'Private Referral Partner (example, L&I)',
    'Multi-Service Center':'Private Referral Partner (example, L&I)',
    'Tacoma Rescue Mission':'Private Referral Partner (example, L&I)',
    'Neighborhood House':'Private Referral Partner (example, L&I)',
    'Ray of Hope':'Private Referral Partner (example, L&I)',
    'Thunderbird Treatment Center':'Private Referral Partner (example, L&I)',
    'Catholic Community Services':'Private Referral Partner (example, L&I)',
    'WorkStep':'Private Referral Partner (example, L&I)',
    'SJI (Seattle Jobs Initiative)':'Private Referral Partner (example, L&I)',
    'Goodwill':'Private Referral Partner (example, L&I)',
    'TRAC Associates':'Private Referral Partner (example, L&I)',
    'Vine Maple Place':'Private Referral Partner (example, L&I)',
    'Partners in Employment':'Private Referral Partner (example, L&I)',
    'Pioneer Industries':'Private Referral Partner (example, L&I)',
    'Auburn School District':'School/Education Program',
    'Snohomish County Human Services | Housing and Community Services':'Other Government Organization',
    'LCSNW - Lutheran Services of the Northwest':'Housing/Shelter Program',
    'We Care Daily Clinics':'Sobriety Program',
    'King County Sheriff Office':'Other Government Organization',
    'Alcoholics Anonymous (AA)':'Sobriety Program',
    'Auburn Community Resource Center':'Other Government Organization',
    'King County Jobs and Housing':'Housing/Shelter Program',
    'Everett Recovery Cafe':'Sobriety Program',
    'Federal Way Courts':'Other Government Organization',
    'Recovery Navigator Program':'Sobriety Program',
    'Take The Next Step':'Sobriety Program',
    'Senior Housing Stability Program':'Housing/Shelter Program',
    'Veterans Administration-Seattle':'Other Government Organization',
    'Veterans Administration-Lakewood':'Other Government Organization',
    'Department of Corrections':'Other Government Organization',
    'Low Income Housing Institute (LIHI)':'Housing/Shelter Program',
    'Battlefield Addiction':'Sobriety Program',
    'Social Media (ex. Facebook, website, etc.)':'Other',
    'Cocoon House':'Housing/Shelter Program'
}

intake3['Referral Source'] = intake3['Referral Source'].replace(ref2rep)

In [308]:
intake3['Guardian: Status'] = intake3['Guardian: Status'].fillna('No')
intake3['Site'] = intake3['Site'].fillna('Auburn')
intake3['Zip Code:County'] = intake3['Zip Code:County'].fillna('King')

In [309]:
intake3['Status'].value_counts()

Status
Started                 1114
Withdrawn                512
Tour Only                236
Denied                   189
Pending / In Process     112
Approved                  44
On Hold                    1
Name: count, dtype: int64

In [310]:
intake4 = intake3[intake3['Status'] != 'Started'].reset_index().copy()

status1list = ['Tour Only','Denied','Withdrawn']
status1 = intake4[(intake4['Status'].isin(status1list)) & (intake4['Date'].dt.year == 2024)]

status2list = ['Approved','On Hold','Pending / In Process']
status2 = intake4[(intake4['Status'].isin(status2list))]

In [311]:
intake5 = pd.concat([status1,status2], ignore_index=True)
intake5 = intake5.copy()
intake5 = intake5.drop(columns='index')
intake5['migrationID'] = range(1,len(intake5) + 1)
intake5['Assigned Staff'] = intake5['Assigned Staff'].str.replace('Jeri Lucas Audit','Jeri Lucas')

In [312]:
pattern = r'\b\d{1,3};#'
intake5['Agency Name'] = intake5['Agency Name'].str.replace(pattern,'',regex=True)

In [313]:
people = merge6[['people_First Name','people_Last Name','demo_Date of Birth']].copy()

In [314]:
intake6 = intake5.merge(people,how='left',left_on=['First Name','Last Name'],right_on=['people_First Name','people_Last Name'])

In [315]:
intake6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 35 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   Assigned Staff                                                                466 non-null    object        
 1   Date Assigned                                                                 466 non-null    datetime64[ns]
 2   Closing Referral without Moving Forward?                                      131 non-null    object        
 3   Date Closed                                                                   245 non-null    datetime64[ns]
 4   Referral Closed Reason                                                        131 non-null    object        
 5   Processing Notes                                                              360 non-null  

In [377]:
intake6.to_excel(r'redacted file path\exports\real intake application.xlsx')

### linking IDs for employers

In [316]:
orgid = pd.read_excel(r'redacted file path\exports\external linking\org id.xlsx')
placementid = pd.read_excel(r'redacted file path\exports\external linking\placement id.xlsx')

placemerge = placementid.merge(orgid,how='left',left_on='Employer Organization Name',right_on='Organization Name')

In [317]:
placemerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Employer Organization Name    282 non-null    object 
 1   employer placement Record ID  282 non-null    int64  
 2   Organization Name             280 non-null    object 
 3   partner org Record ID         280 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 8.9+ KB


In [688]:
placemerge.to_excel(r'redacted file path\exports\placement organization linking ids2.xlsx')

In [318]:
detail = pd.read_excel(r'redacted file path\exports\external linking\placement org dedupe ids.xlsx')

In [319]:
detail2 = detail[detail['Employer Organization Name'] == detail['import employer name']]

In [690]:
detail2.to_excel(r'redacted file path\exports\external linking\placement org dedupe ids DEDUPED.xlsx')

### Linking goals by id

In [320]:
goaldate = pd.read_excel(r'redacted file path\exports\external linking\service plan to goal linking by date.xlsx')

In [321]:
goaldate = goaldate.sort_values(by=['Date Goal Opened','sub service date'],ascending=True)

In [322]:
goaldate_equal = goaldate[goaldate['Date Goal Opened'] == goaldate['sub service date']].copy()
goaldate_unequal = goaldate[goaldate['Date Goal Opened'] != goaldate['sub service date']].copy()

In [323]:
# goaldate_equal.to_excel(r'redacted file path\exports\external linking\goal service equal DEDUPED.xlsx')

In [324]:
goaldate_unequal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 604 entries, 711 to 1305
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Record ID                 604 non-null    object
 1   service plan Record ID    604 non-null    int64 
 2   Service text calc         604 non-null    object
 3   sub service date          604 non-null    object
 4   Date Goal Opened          604 non-null    object
 5   goal Record ID            604 non-null    int64 
 6   Import service text calc  604 non-null    object
dtypes: int64(2), object(5)
memory usage: 37.8+ KB


In [325]:
# Filter to ensure 'Date Goal Opened' is on or after 'sub service date'
filtered_goaldate = goaldate_unequal[goaldate_unequal['Date Goal Opened'] >= goaldate_unequal['sub service date']]

# Remove duplicates based on 'goal Record ID' while keeping the first occurrence
goaldate2 = filtered_goaldate.drop_duplicates(subset='goal Record ID', keep='first')

In [326]:
goaldate3 = pd.concat([goaldate_equal,goaldate2])
goaldate3 = goaldate3.drop_duplicates(subset='goal Record ID', keep='first')

In [327]:
goaldate3 = goaldate3.sort_values(by='Record ID')

In [731]:
goaldate3.to_excel(r'redacted file path\exports\external linking\goal service DEDUPED.xlsx')